# **Create Event Data**

In [15]:
# Importing dask and pandas 
import dask.dataframe as dd # for large datasets
import pandas as pd


In [16]:
# Loading the events data from events.csv file using dask
df_event = dd.read_csv('/home/nkama/masters_thesis_project/thesis/data/event_rec_engine_challenge/events.csv',
                 dtype={
                     'event_id': 'object',
                     'city': 'object',
                     'country': 'object',
                     'state': 'object',
                     'zip': 'object'
                 })

In [17]:
df_event.head()

,event_id,user_id,start_time,city,state,zip,country,lat,lng,c_1,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,684921758,3647864012,2012-10-31T00:00:00.001Z,<NA>,<NA>,<NA>,<NA>,NaN,NaN,2,...,0,1,0,0,0,0,0,0,0,9
1,244999119,3476440521,2012-11-03T00:00:00.001Z,<NA>,<NA>,<NA>,<NA>,NaN,NaN,2,...,0,0,0,0,0,0,0,0,0,7
2,3928440935,517514445,2012-11-05T00:00:00.001Z,<NA>,<NA>,<NA>,<NA>,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,12
3,2582345152,781585781,2012-10-30T00:00:00.001Z,<NA>,<NA>,<NA>,<NA>,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,8
4,1051165850,1016098580,2012-09-27T00:00:00.001Z,<NA>,<NA>,<NA>,<NA>,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,9


The idea here is to select from the event dataframe events that are also find in the
attendees data file. This is to ensure that we collect events with records of some interactions (attendance information) with users.

Since the dataset is very large, we can work with only the first 9 columns of the event data for now to reduce computational cost. Unfortunately, the data does't have event title and description which we also need int the project and so we will use the rest of the columns which are bag of words representation of the events title and descriptions will be used later on to synthesize the event title and descriptions.

In [8]:
# loading the first 9 columns of the event data
new_event_df = df_event.iloc[:,:9]
new_event_df.head()


,event_id,user_id,start_time,city,state,zip,country,lat,lng
0,684921758,3647864012,2012-10-31T00:00:00.001Z,<NA>,<NA>,<NA>,<NA>,NaN,NaN
1,244999119,3476440521,2012-11-03T00:00:00.001Z,<NA>,<NA>,<NA>,<NA>,NaN,NaN
2,3928440935,517514445,2012-11-05T00:00:00.001Z,<NA>,<NA>,<NA>,<NA>,NaN,NaN
3,2582345152,781585781,2012-10-30T00:00:00.001Z,<NA>,<NA>,<NA>,<NA>,NaN,NaN
4,1051165850,1016098580,2012-09-27T00:00:00.001Z,<NA>,<NA>,<NA>,<NA>,NaN,NaN


In [7]:
len(new_event_df) # check the length


3137972

In [12]:
# Load the interactions data to be used to filter events fromt the event data.

user_event_pairs = pd.read_csv("/home/nkama/masters_thesis_project/thesis/interactions.csv")

user_event_pairs.head()


,Unnamed: 0,event_id,user_id,attendance_status
0,0,1159822043,1975964455,yes
1,1,1159822043,252302513,yes
2,2,1159822043,4226086795,yes
3,3,1159822043,3805886383,yes
4,4,1159822043,1420484491,yes


In [13]:
# Extract unique event_ids and user_ids
unique_events = user_event_pairs['event_id'].unique()

# Print the number of unique events 
print("Number of unique events:", len(unique_events))

# Check data types
print("Data types in user_event_pairs:")
print(user_event_pairs['event_id'].dtype)
print("\nData types in new_event_df:")
print(new_event_df['event_id'].dtype)

Number of unique events: 22710
Data types in user_event_pairs:
int64

Data types in new_event_df:
string


In [14]:

# Convert both to string type to ensure matching
user_event_pairs['event_id'] = user_event_pairs['event_id'].astype(str)
new_event_df['event_id'] = new_event_df['event_id'].astype(str)


In [ ]:
# Filter events data
filtered_events_data = df_event[df_event["event_id"].isin(unique_events)].compute()
filtered_events_data.head()


In [19]:
filtered_events_data = pd.read_csv("/home/nkama/masters_thesis_project/thesis/filtered_events_data.csv")
filtered_events_data.head()

,Unnamed: 0.1,Unnamed: 0,event_id,start_time,city,lat,lng,c_1,c_2,c_3,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,40,40,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,0,3,1,...,0,0,0,0,0,0,0,0,0,28
1,51,51,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,5,3,3,...,0,0,0,0,1,0,0,0,0,103
2,74,74,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,5,3,3,...,0,0,0,0,1,0,0,0,0,103
3,155,156,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,0,0,0,...,0,0,0,0,0,0,0,0,0,6
4,177,178,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Print the shape of filtered data
#print("\nOriginal events shape:", new_event_df.shape[0].compute())
print("Filtered events shape:", filtered_events_data.shape)

Filtered events shape: (11627, 108)


In [15]:
# Show counts of matching events
print("\nNumber of unique events:", len(unique_events))
#print("Number of events in filtered data:", len(filtered_events_data))
print("Number of matching events:", len(set(unique_events).intersection(set(filtered_events_data['event_id']))))


Number of unique events: 22710
Number of matching events: 22471


In [16]:
# Check for missing values in columns
filtered_events_data[["city",	"state",	"zip",	"country",	"lat",	"lng"]].isnull().sum()

city       10782
state      13540
zip        18941
country    10711
lat         8135
lng         8135
dtype: int64

In [ ]:
#  Drop irrelevant columns 
filtered_events_data.drop(columns=(["user_id", "state",	"zip",	"country"]), inplace=True)
print(len(filtered_events_data))
filtered_events_data.head()

22471


,Unnamed: 0,event_id,start_time,city,lat,lng,c_1,c_2,c_3,c_4,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,0,684921758,2012-10-31T00:00:00.001Z,NaN,NaN,NaN,2,0,2,0,...,0,1,0,0,0,0,0,0,0,9
1,1,244999119,2012-11-03T00:00:00.001Z,NaN,NaN,NaN,2,0,2,0,...,0,0,0,0,0,0,0,0,0,7
2,2,3928440935,2012-11-05T00:00:00.001Z,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
3,3,2582345152,2012-10-30T00:00:00.001Z,NaN,NaN,NaN,1,0,2,1,...,0,0,0,0,0,0,0,0,0,8
4,4,1051165850,2012-09-27T00:00:00.001Z,NaN,NaN,NaN,1,1,0,0,...,0,0,0,0,0,0,0,0,0,9


In [21]:
# Drop missing values.
filtered_events_data.dropna(subset=["city",	"lat",	"lng"], inplace=True)
len(filtered_events_data)

11627

In [22]:
# Save filtered events data to csv
filtered_events_data.to_csv("filtered_events_data.csv")

In [24]:
# event_derails script to generate event titlw, description and category.
from event_details import synthesize_event_details, add_variety

In [25]:
#events_df = load_events_data('/content/large_col_events.csv')
enriched_events = synthesize_event_details(filtered_events_data)
enriched_events = add_variety(enriched_events)


Processed 3000 events
Processed 6000 events
Processed 7000 events
Processed 8000 events
Processed 9000 events
Processed 10000 events
Processed 12000 events
Processed 13000 events
Processed 14000 events
Processed 18000 events
Processed 21000 events
Processed 22000 events


In [27]:
enriched_events.head(3)

,Unnamed: 0,event_id,start_time,city,lat,lng,c_1,c_2,c_3,c_4,...,c_95,c_96,c_97,c_98,c_99,c_100,c_other,category,title,description
40,40,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,0,3,1,1,...,0,0,0,0,0,0,28,Technology,Smart {tech_area} Session,We're excited to bring you this Summer meetup ...
51,51,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,5,3,3,1,...,0,1,0,0,0,0,103,Arts & Culture,Palo Alto {art_form} Collection,We're excited to bring you this Summer install...
74,74,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,5,3,3,1,...,0,1,0,0,0,0,103,Arts & Culture,Karachi {art_form} Museum,We're excited to bring you this Weekend showin...


In [29]:
enriched_events.to_csv('enriched_events.csv', index=False)

In [28]:
len(enriched_events)


11627

In [32]:
enriched_events = pd.read_csv("/home/nkama/masters_thesis_project/thesis/enriched_events.csv")
enriched_events.isnull().sum().sum()

np.int64(0)

In [33]:

enriched_events_col = ["event_id",	"start_time",	"city",	"lat",	"lng",	"category",	"title",	"description"]
enriched_events = enriched_events[enriched_events_col]
enriched_events.head()


,event_id,start_time,city,lat,lng,category,title,description
0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,Technology,Smart {tech_area} Session,We're excited to bring you this Summer meetup ...
1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,Arts & Culture,Palo Alto {art_form} Collection,We're excited to bring you this Summer install...
2,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,Arts & Culture,Karachi {art_form} Museum,We're excited to bring you this Weekend showin...
3,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,Arts & Culture,{art_form} Gallery: Traditional Feature,We're excited to bring you this Weekend displa...
4,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,Entertainment,Live {genre} Event,Be part of for this featured tour in Palo Alto...


In [34]:
enriched_events.to_csv("events_with_titles.csv")


## Merge with attendees count


In [ ]:
#load attendee counts df
attendance_counts = pd.read_csv("attendance_counts.csv")
attendance_counts.head()

In [ ]:

# Drop the 'Unnamed: 0' column
#enriched_events2 = attendance_counts.drop(columns=['Unnamed: 0'])

attendance_counts = attendance_counts.rename(columns={'event': 'event_id'})
attendance_counts['event_id'] = attendance_counts['event_id'].astype('object')


# Merge the DataFrames with inner join to keep only common events
events_data = enriched_events.merge(
    attendance_counts,
    on='event_id',
    how='inner'
)


In [ ]:

len(events_data)


In [ ]:
events_data.head()


In [ ]:
columns = ["event_id",	"start_time",	"city",	"lat",	"lng", "category",
                       "title",	"description",	"yes_count",	"maybe_count",	"invited_count",
                       "no_count",	"total_users"]
events_data = events_data[columns]
events_data.head()


In [ ]:
events_data.to_csv("events_data.csv") # Save data to csv.


## Add Weather data to the event data


In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests


In [3]:
# loading the saved events data
events_data = pd.read_csv("/home/nkama/masters_thesis_project/thesis/events_data.csv").drop(columns=["Unnamed: 0"])
print(len(events_data))
events_data.head()

11627


,event_id,start_time,city,lat,lng,category,title,description,yes_count,maybe_count,invited_count,no_count,total_users
0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,Sports & Fitness,Sihanoukville {sport} Challenge,Join the regional {sport} community for our Ho...,93,65,317,47,522
1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,Music & Concerts,{genre} Album: Holiday Series,Join us for an indie {genre} festival in Palo ...,162,112,1021,150,1445
2,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,Business & Networking,Karachi {field} Meetup,Don't miss our industry conference happening i...,22,8,993,26,1049
3,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,Health & Wellness,{practice} Class: Healing Strengthening,Don't miss our healing therapy happening in Lo...,63,42,430,20,555
4,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,Sports & Fitness,Fall {sport} League,Challenge yourself at the Palo Alto {sport} co...,41,12,53,1,107


To get weather data, we have to make api calls to the Open-Meteo weather API. But first, we split the events data into three. this is to ensure that we meet the rate limit of max 5000 calls within an hour.


In [6]:
# importing the script with functions to split dataframe, retreive weather info and concat all splits  
from weather_data import split_eventsdf_into_three, get_weather_for_events, concat_all_splits

In [6]:
# Split the event_data into three
first_split, second_split, last_split = split_eventsdf_into_three(events_data)
len(first_split), len(second_split), len(last_split)

(3875, 3875, 3877)

In [1]:
# Confirm complete length (11627)
3875+ 3875+ 3877


11627

In [39]:
first_split.head(3)

,event_id,start_time,city,lat,lng,category,title,description,yes_count,maybe_count,invited_count,no_count,total_users
0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,Sports & Fitness,Sihanoukville {sport} Challenge,Join the regional {sport} community for our Ho...,93,65,317,47,522
1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,Music & Concerts,{genre} Album: Holiday Series,Join us for an indie {genre} festival in Palo ...,162,112,1021,150,1445
2,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,Business & Networking,Karachi {field} Meetup,Don't miss our industry conference happening i...,22,8,993,26,1049


,Unnamed: 0,event_id,start_time,city,lat,lng,category,title,description,yes_count,maybe_count,invited_count,no_count,total_users,weather_code,temperature_2m_mean,precipitation_sum,precipitation_hours,wind_speed_10m_max
0,0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,Sports & Fitness,Sihanoukville {sport} Challenge,Join the regional {sport} community for our Ho...,93,65,317,47,522,51.0,28.318502,0.100000,1.0,10.972620
1,1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,Music & Concerts,{genre} Album: Holiday Series,Join us for an indie {genre} festival in Palo ...,162,112,1021,150,1445,3.0,18.417002,0.000000,0.0,9.826088
2,2,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,Business & Networking,Karachi {field} Meetup,Don't miss our industry conference happening i...,22,8,993,26,1049,3.0,28.837250,0.000000,0.0,30.421598
3,3,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,Health & Wellness,{practice} Class: Healing Strengthening,Don't miss our healing therapy happening in Lo...,63,42,430,20,555,61.0,24.851418,7.200001,12.0,6.989936
4,4,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,Sports & Fitness,Fall {sport} League,Challenge yourself at the Palo Alto {sport} co...,41,12,53,1,107,3.0,10.835751,0.000000,0.0,10.495713


Now we run the get_weather_for_events function to retrieve weather data for each event from all three dataframe splits.


In [ ]:
# Get weather information for the first split
first_split_weather = get_weather_for_events(first_split)
first_split_weather.head()

In [ ]:
# Save first split with weather information to csv
first_split_weather.to_csv("first_split_weather.csv")

In [40]:
# Get weather information for the second split
second_split_weather = get_weather_for_events(second_split)
second_split_weather.to_csv("second_split_weather.csv")

Processing batch 1/78
Processing batch 2/78
Processing batch 3/78
Processing batch 4/78
Processing batch 5/78
Processing batch 6/78
Processing batch 7/78
Processing batch 8/78
Processing batch 9/78
Processing batch 10/78
Processing batch 11/78
Processing batch 12/78
Processing batch 13/78
Processing batch 14/78
Processing batch 15/78
Processing batch 16/78
Processing batch 17/78
Processing batch 18/78
Processing batch 19/78
Processing batch 20/78
Processing batch 21/78
Processing batch 22/78
Processing batch 23/78
Processing batch 24/78
Processing batch 25/78
Processing batch 26/78
Processing batch 27/78
Processing batch 28/78
Processing batch 29/78
Processing batch 30/78
Processing batch 31/78
Processing batch 32/78
Processing batch 33/78
Processing batch 34/78
Processing batch 35/78
Processing batch 36/78
Processing batch 37/78
Processing batch 38/78
Processing batch 39/78
Processing batch 40/78
Processing batch 41/78
Processing batch 42/78
Processing batch 43/78
Processing batch 44/

: 

In [8]:
# Get weather information for the last split
last_split_weather = get_weather_for_events(last_split)
# Save last split to a csv file
last_split_weather.to_csv("last_split_weather.csv")

Processing batch 1/78
Processing batch 2/78
Processing batch 3/78
Processing batch 4/78
Processing batch 5/78
Processing batch 6/78
Processing batch 7/78
Processing batch 8/78
Processing batch 9/78
Processing batch 10/78
Processing batch 11/78
Processing batch 12/78
Processing batch 13/78
Processing batch 14/78
Processing batch 15/78
Processing batch 16/78
Processing batch 17/78
Processing batch 18/78
Processing batch 19/78
Processing batch 20/78
Processing batch 21/78
Processing batch 22/78
Processing batch 23/78
Processing batch 24/78
Processing batch 25/78
Processing batch 26/78
Processing batch 27/78
Processing batch 28/78
Processing batch 29/78
Processing batch 30/78
Processing batch 31/78
Processing batch 32/78
Processing batch 33/78
Processing batch 34/78
Processing batch 35/78
Processing batch 36/78
Processing batch 37/78
Processing batch 38/78
Processing batch 39/78
Processing batch 40/78
Processing batch 41/78
Processing batch 42/78
Processing batch 43/78
Processing batch 44/

In [12]:
first_split_weather = pd.read_csv("/home/nkama/masters_thesis_project/thesis/first_split_weather.csv")
second_split_weather = pd.read_csv("/home/nkama/masters_thesis_project/thesis/second_split_weather.csv")

In [13]:
# Concatenate all splits with the weather information
complete_event_data = concat_all_splits(first_split_weather,second_split_weather,last_split_weather)
complete_event_data.head()


,Unnamed: 0,event_id,start_time,city,lat,lng,category,title,description,yes_count,maybe_count,invited_count,no_count,total_users,weather_code,temperature_2m_mean,precipitation_sum,precipitation_hours,wind_speed_10m_max
0,0.0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,Sports & Fitness,Sihanoukville {sport} Challenge,Join the regional {sport} community for our Ho...,93,65,317,47,522,51.0,28.318502,0.1,1.0,10.97262
1,1.0,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,Music & Concerts,{genre} Album: Holiday Series,Join us for an indie {genre} festival in Palo ...,162,112,1021,150,1445,3.0,18.417002,0.0,0.0,9.826088
2,2.0,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,Business & Networking,Karachi {field} Meetup,Don't miss our industry conference happening i...,22,8,993,26,1049,3.0,28.83725,0.0,0.0,30.421598
3,3.0,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,Health & Wellness,{practice} Class: Healing Strengthening,Don't miss our healing therapy happening in Lo...,63,42,430,20,555,61.0,24.851418,7.200001,12.0,6.989936
4,4.0,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,Sports & Fitness,Fall {sport} League,Challenge yourself at the Palo Alto {sport} co...,41,12,53,1,107,3.0,10.835751,0.0,0.0,10.495713


In [14]:
len(complete_event_data)

11627

In [7]:

first = pd.read_csv("/home/nkama/masters_thesis_project/thesis/first_split_weather.csv")
second = pd.read_csv("/home/nkama/masters_thesis_project/thesis/second_split_weather.csv")
third = pd.read_csv("/home/nkama/masters_thesis_project/thesis/last_split_weather.csv")

# Concatenate all splits with the weather information
complete_event_data = concat_all_splits(first,second,third)

In [11]:
complete_event_data.drop(columns=(["Unnamed: 0", "category", "title",	"description"]), inplace = True)
complete_event_data.head()


,event_id,start_time,city,lat,lng,yes_count,maybe_count,invited_count,no_count,total_users,weather_code,temperature_2m_mean,precipitation_sum,precipitation_hours,wind_speed_10m_max
0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,93,65,317,47,522,51.0,28.318502,0.100000,1.0,10.972620
1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,162,112,1021,150,1445,3.0,18.417002,0.000000,0.0,9.826088
2,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,22,8,993,26,1049,3.0,28.837250,0.000000,0.0,30.421598
3,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,63,42,430,20,555,61.0,24.851418,7.200001,12.0,6.989936
4,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,41,12,53,1,107,3.0,10.835751,0.000000,0.0,10.495713


In [12]:
complete_event_data.to_csv("event_data_category_gen.csv")

In [ ]:
events_data_weather = pd.read_csv("/home/nkama/masters_thesis_project/thesis/event_data_category_gen.csv")\
    .drop(columns=["Unnamed: 0"])

filtered_events_data = pd.read_csv("/home/nkama/masters_thesis_project/thesis/filtered_events_data.csv")\
    .drop(columns=["Unnamed: 0.1", "Unnamed: 0"])

filtered_events_data.head()

,event_id,start_time,city,lat,lng,c_1,c_2,c_3,c_4,c_5,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,0,3,1,1,1,...,0,0,0,0,0,0,0,0,0,28
1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,5,3,3,1,3,...,0,0,0,0,1,0,0,0,0,103
2,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,5,3,3,1,3,...,0,0,0,0,1,0,0,0,0,103
3,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
4,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
events_data_weather.head()

,event_id,start_time,city,lat,lng,yes_count,maybe_count,invited_count,no_count,total_users,weather_code,temperature_2m_mean,precipitation_sum,precipitation_hours,wind_speed_10m_max
0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,93,65,317,47,522,51.0,28.318502,0.100000,1.0,10.972620
1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,162,112,1021,150,1445,3.0,18.417002,0.000000,0.0,9.826088
2,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,22,8,993,26,1049,3.0,28.837250,0.000000,0.0,30.421598
3,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,63,42,430,20,555,61.0,24.851418,7.200001,12.0,6.989936
4,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,41,12,53,1,107,3.0,10.835751,0.000000,0.0,10.495713


In [32]:
print(len(events_data_weather))
len(filtered_events_data)

11627


11627

In [33]:
big_events_data = filtered_events_data.merge(events_data_weather,
                                             on=["event_id","start_time","city","lat","lng"],
                                             how="inner")
print(len(big_events_data))
big_events_data.head()

11627


,event_id,start_time,city,lat,lng,c_1,c_2,c_3,c_4,c_5,...,yes_count,maybe_count,invited_count,no_count,total_users,weather_code,temperature_2m_mean,precipitation_sum,precipitation_hours,wind_speed_10m_max
0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,0,3,1,1,1,...,93,65,317,47,522,51.0,28.318502,0.100000,1.0,10.972620
1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,5,3,3,1,3,...,162,112,1021,150,1445,3.0,18.417002,0.000000,0.0,9.826088
2,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,5,3,3,1,3,...,22,8,993,26,1049,3.0,28.837250,0.000000,0.0,30.421598
3,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,0,0,0,0,0,...,63,42,430,20,555,61.0,24.851418,7.200001,12.0,6.989936
4,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,0,0,0,0,0,...,41,12,53,1,107,3.0,10.835751,0.000000,0.0,10.495713


In [110]:
big_events_data.columns

Index(['event_id', 'start_time', 'city', 'lat', 'lng', 'c_1', 'c_2', 'c_3',
       'c_4', 'c_5',
       ...
       'yes_count', 'maybe_count', 'invited_count', 'no_count', 'total_users',
       'weather_code', 'temperature_2m_mean', 'precipitation_sum',
       'precipitation_hours', 'wind_speed_10m_max'],
      dtype='object', length=116)

In [35]:
big_events_data.shape, filtered_events_data.shape, events_data_weather.shape

((11627, 116), (11627, 106), (11627, 15))

In [98]:
import math
import random
import pandas as pd
from datetime import datetime
from litellm import completion


# Weather interpretation codes
WEATHER_CODES = {
    0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
    45: "Fog", 48: "Depositing rime fog", 51: "Light drizzle", 53: "Moderate drizzle",
    55: "Dense drizzle", 56: "Light freezing drizzle", 57: "Dense freezing drizzle",
    61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain", 66: "Light freezing rain",
    67: "Heavy freezing rain", 71: "Slight snow fall", 73: "Moderate snow fall",
    75: "Heavy snow fall", 77: "Snow grains", 80: "Slight rain showers",
    81: "Moderate rain showers", 82: "Violent rain showers", 85: "Slight snow showers",
    86: "Heavy snow showers", 95: "Thunderstorm", 96: "Thunderstorm with slight hail",
    99: "Thunderstorm with heavy hail"
}

EVENT_CATEGORIES = [
    'Music & Concerts', 'Food & Drink', 'Education & Learning', 'Sports & Fitness',
    'Arts & Culture', 'Business & Networking', 'Technology', 'Community & Causes',
    'Health & Wellness', 'Entertainment', 'Seasonal & Festivals', 'Immersive Experiences'
]

# Helper functions
def parse_datetime(time_value):
    """Parse datetime from various formats"""
    if pd.isna(time_value):
        return None
    try:
        if isinstance(time_value, str):
            return datetime.fromisoformat(time_value.replace('Z', '+00:00'))
        return time_value
    except (ValueError, AttributeError):
        return None

def format_time(time_value):
    """Format datetime to readable string"""
    dt = parse_datetime(time_value)
    if dt:
        return dt.strftime("%B %d, %Y at %I:%M %p")
    return "TBD"

def calculate_weather_bias(event_row):
    """Calculate how much the weather pushes events indoors (0-1 scale)"""
    bias = 0
    
    # Weather code check
    weather_code = event_row.get('weather_code')
    if weather_code is not None and not pd.isna(weather_code):
        # Precipitation or extreme weather codes increase indoor bias
        if weather_code in [3, 45, 48, 51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 71, 73, 75, 77, 80, 81, 82, 85, 86, 95, 96, 99]:
            bias = max(bias, 0.6)
    
    # Temperature extremes
    temperature = event_row.get('temperature_2m_mean')
    if temperature is not None and not pd.isna(temperature):
        if temperature < 10 or temperature > 30:
            bias = max(bias, 0.5)
    
    # Heavy precipitation
    precipitation = event_row.get('precipitation_sum')
    if precipitation is not None and not pd.isna(precipitation) and precipitation > 5:
        bias = max(bias, 0.7)
    
    # Strong winds
    wind_speed = event_row.get('wind_speed_10m_max')
    if wind_speed is not None and not pd.isna(wind_speed) and wind_speed > 20:
        bias = max(bias, 0.6)
        
    return bias

def categorize_by_words(word_counts, is_evening, is_weekend, weather_indoor_bias):
    """Determine category based on word patterns"""
    total_words = sum(word_counts)
    num_unique_words = sum(1 for count in word_counts if count > 0)
    max_count = max(word_counts) if word_counts else 0
    concentration = max_count / total_words if total_words > 0 else 0
    
    # High word diversity suggests educational/business events
    if num_unique_words > 15 and concentration < 0.15:
        if is_evening:
            return random.choice(['Business & Networking', 'Technology'])
        else:
            return random.choice(['Education & Learning', 'Business & Networking', 'Technology'])
    
    # High concentration suggests focused events
    if concentration > 0.3:
        if is_evening:
            if weather_indoor_bias > 0.5:
                return random.choice(['Entertainment', 'Arts & Culture'])
            else:
                return random.choice(['Music & Concerts', 'Entertainment'])
        else:
            if weather_indoor_bias > 0.5:
                return random.choice(['Arts & Culture', 'Education & Learning'])
            else:
                return random.choice(['Sports & Fitness', 'Community & Causes'])
    
    # Medium concentration with few words often suggests food or social events
    if 0.2 <= concentration <= 0.3 and num_unique_words < 10:
        if is_evening:
            return random.choice(['Food & Drink', 'Entertainment'])
        else:
            return random.choice(['Food & Drink', 'Health & Wellness'])
    
    # Default to weighted selection
    return weighted_category_selection(weather_indoor_bias)

def categorize_by_time_weather(is_evening, is_weekend, weather_indoor_bias):
    """Category determination based on time and weather when word data unavailable"""
    if is_evening:
        if weather_indoor_bias > 0.5:
            return random.choice(['Entertainment', 'Arts & Culture', 'Food & Drink'])
        else:
            return random.choice(['Music & Concerts', 'Entertainment', 'Food & Drink'])
    elif is_weekend:
        if weather_indoor_bias > 0.5:
            return random.choice(['Arts & Culture', 'Education & Learning', 'Health & Wellness'])
        else:
            return random.choice(['Sports & Fitness', 'Community & Causes', 'Seasonal & Festivals'])
    else:
        return random.choice(['Business & Networking', 'Technology', 'Education & Learning'])

def weighted_category_selection(weather_indoor_bias):
    """Select category based on weights adjusted for weather"""
    # Base weights
    weights = [0.15, 0.12, 0.08, 0.08, 0.10, 0.08, 0.06, 0.08, 0.08, 0.07, 0.05, 0.05]
    
    # Adjust for weather
    if weather_indoor_bias > 0.5:
        # Indices for indoor-friendly events
        indoor_indices = [1, 2, 4, 5, 6, 8, 9]  
        # Indices for outdoor events
        outdoor_indices = [0, 3, 7, 10, 11]
        
        # Boost indoor categories, reduce outdoor ones
        for idx in indoor_indices:
            weights[idx] *= 1.3
        for idx in outdoor_indices:
            weights[idx] *= 0.7
            
        # Normalize weights
        total = sum(weights)
        weights = [w/total for w in weights]
    
    return random.choices(EVENT_CATEGORIES, weights=weights, k=1)[0]

def determine_category(event_row):
    """Determine the most likely event category based on available data"""
    # Extract word count data
    word_counts = [event_row.get(col, 0) for col in event_row.index if col.startswith('c_') and col != 'c_other']
    word_counts = [count for count in word_counts if not pd.isna(count)]
    
    # Calculate attendance metrics
    yes_count = event_row.get('yes_count', 0)
    maybe_count = event_row.get('maybe_count', 0)
    invited_count = event_row.get('invited_count', 0)
    is_large_event = invited_count > 500 if not pd.isna(invited_count) else False
    
    # Parse datetime
    dt = parse_datetime(event_row.get('start_time'))
    is_evening = dt and dt.hour >= 18
    is_weekend = dt and dt.weekday() >= 5
    month = dt.month if dt else None
    
    # Calculate weather influence
    weather_indoor_bias = calculate_weather_bias(event_row)
    
    # Check for seasonal events first
    if month and random.random() < (0.4 if month in [11, 12] else 0.1):
        return 'Seasonal & Festivals'
        
    # Handle large events differently
    if is_large_event:
        if weather_indoor_bias < 0.3:
            return random.choice(['Music & Concerts', 'Sports & Fitness', 'Immersive Experiences'])
        else:
            return random.choice(['Technology', 'Entertainment', 'Business & Networking'])
    
    # Analyze word patterns if available
    if word_counts:
        return categorize_by_words(word_counts, is_evening, is_weekend, weather_indoor_bias)
    
    # Default categorization based on time and weather
    return categorize_by_time_weather(is_evening, is_weekend, weather_indoor_bias)

def format_weather_context(event_row):
    """Format weather data into readable context"""
    weather_code = event_row.get('weather_code')
    temperature = event_row.get('temperature_2m_mean')
    precipitation = event_row.get('precipitation_sum')
    wind_speed = event_row.get('wind_speed_10m_max')
    
    if weather_code is None or pd.isna(weather_code):
        return ""
        
    weather_desc = WEATHER_CODES.get(weather_code, '')
    
    weather_context = f"Weather forecast: {weather_desc}"
    
    if temperature is not None and not pd.isna(temperature):
        weather_context += f", {temperature:.1f}°C"
    
    if precipitation is not None and not pd.isna(precipitation) and precipitation > 0:
        weather_context += f", with {precipitation:.1f}mm precipitation expected"
    
    if wind_speed is not None and not pd.isna(wind_speed) and wind_speed > 10:
        weather_context += f", and winds up to {wind_speed:.1f}km/h"
        
    return weather_context

def get_location_string(event_row):
    """Format location data into readable string"""
    city = event_row.get('city', 'Local')
    state = event_row.get('state', '')
    return f"{city}, {state}" if state else city

def determine_event_setting(category, event_row, weather_desc):
    """Determine if an event is indoor or outdoor"""
    # Categories that are typically outdoor
    outdoor_categories = [
        'Sports & Fitness', 'Music & Concerts', 
        'Seasonal & Festivals', 'Community & Causes', 
        'Immersive Experiences'
    ]
    
    # Extract weather conditions
    weather_code = event_row.get('weather_code')
    precipitation = event_row.get('precipitation_sum')
    wind_speed = event_row.get('wind_speed_10m_max')
    temperature = event_row.get('temperature_2m_mean')
    
    # Check if weather is unsuitable for outdoor events
    bad_weather = False
    if weather_code is not None and not pd.isna(weather_code):
        if weather_code in [51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 71, 73, 75, 77, 80, 81, 82, 85, 86, 95, 96, 99]:
            bad_weather = True
    
    if precipitation is not None and not pd.isna(precipitation) and precipitation > 5:
        bad_weather = True
    
    if wind_speed is not None and not pd.isna(wind_speed) and wind_speed > 20:
        bad_weather = True
    
    if temperature is not None and not pd.isna(temperature) and (temperature < 5 or temperature > 35):
        bad_weather = True
    
    # Determine event type
    if category in outdoor_categories and not bad_weather:
        return "Outdoor"
    elif category not in outdoor_categories:
        return "Indoor"
    else:
        # This could be an LLM call in a real implementation
        return "Indoor"  # Default for ambiguous cases

def llm_response(prompt, llm_model="gpt-4o-mini"):
    """Function to get response from language model"""
    # This is a placeholder - in real implementation, you would call your LLM API
    # For now, we'll return a placeholder response
    response = completion(  # Assuming completion function is defined elsewhere
        model=llm_model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

def generate_llm_title(category, event_row):
    """Generate a title for an event using LLM"""
    location = get_location_string(event_row)
    
    # Parse datetime
    dt = parse_datetime(event_row.get('start_time'))
    
    # Time and season descriptions
    time_desc, season = '', ''
    if dt:
        hour = dt.hour
        month = dt.month
        
        # Time of day
        if 5 <= hour < 12: time_desc = "morning"
        elif 12 <= hour < 17: time_desc = "afternoon"
        elif 17 <= hour < 21: time_desc = "evening"
        else: time_desc = "night"
            
        # Season (adjusted for hemisphere)
        is_northern = event_row.get('lat', 0) > 0 if 'lat' in event_row and not pd.isna(event_row['lat']) else True
        
        if is_northern:
            if month in [12, 1, 2]: season = "winter"
            elif month in [3, 4, 5]: season = "spring"
            elif month in [6, 7, 8]: season = "summer"
            else: season = "fall"
        else:
            if month in [12, 1, 2]: season = "summer"
            elif month in [3, 4, 5]: season = "fall"
            elif month in [6, 7, 8]: season = "winter"
            else: season = "spring"
    
    # Generate title prompt with added variation

    # List of event title examples for few-shot learning
    events_titles = [
        "Holi In Berlin 2025 - Festival of Colors | Food Stalls | Bollywood Dance",
        "Berlin Chocolate Festival 2025",
        "REGGAE ON THE ROOF",
        "Sing Along New York (Hits der 90er & 2000er), 29.03.2025",
        "Hackaday Europe 2025",
        "Just Taylor - One Night All Eras // Festsaal Kreuzberg",
        "2025 BUSINESS & LEADERSHIP CONFERENCE (BLC)",
        "SPECIAL: Millennials After Dark Professional Networking @ BLVD Park",
        "ASGTG E-Commerce Amazon Sellers Event 11th Annual Ed Rosenberg Conference",
        "San Diego Big Business, Tech & Entrepreneur Professional Networking Soiree",
    ]
        # Format examples to show expected style
    formatted_examples = "\n".join([f"- {title}" for title in events_titles])

    # Updated Prompt
    title_prompt = f"""
    Generate a catchy, natural-sounding event title similar in style to the examples below.

    ### **Examples of Great Event Titles:**
    {formatted_examples}

    ### **Event Details:**
    - Category: {category}
    - Location: {location}
    - Time of day: {time_desc}
    - Season: {season}



    ### **Instructions:**
    - Your title **must resemble the style of the examples**.
    - **Do NOT** use a formulaic "Season + Location + Event" format.
    - Only randomly incorporate season or time of day for some events
    - **Use dynamic, creative, and engaging phrasing**.
    - **Incorporate real-world language & cultural references** like the examples.
    - **No unnecessary punctuation**, no explanations, just return the title.

    ### **Output Format:**
    - **Only the title**
    - **No colons (":") unless absolutely necessary**
    - **Use expressive, engaging language**
    """


    
    title = llm_response(prompt=title_prompt)
    return title.strip().strip('"\'').strip()

def get_event_type(category, event_row):
    """Generate a description for an event using LLM"""
    
    # Format weather information
    weather_context = format_weather_context(event_row)

    # Determine if the event is indoor or outdoor
    event_type = determine_event_setting(category, event_row, weather_context)

    return event_type



In [ ]:
def process_events_dataframe(events_df):
    """
    Process all events in a dataframe using the refactored event categorization system
    
    Parameters:
    -----------
    events_df : pandas.DataFrame
        DataFrame containing event data with columns like city, state, start_time, etc.
    generate_descriptions : bool, default=True
        Whether to generate event descriptions (can be time-consuming with LLM calls)
        
    Returns:
    --------
    pandas.DataFrame
        Original dataframe with additional columns for category, title, and optionally event_type and description
    """
    # Create new columns for our generated data
    result_df = events_df.copy()
    result_df['category'] = None
    result_df['title'] = None
    result_df['event_type'] = None
    #result_df['description'] = None
    
    # Process each event row by row
    total_events = len(result_df)
    
    print(f"Processing {total_events} events...")
    
    for i, (idx, event_row) in enumerate(result_df.iterrows()):
        try:
            # Print progress update every 10 events
            if i % 10 == 0 or i == total_events - 1:
                print(f"Processing event {i+1}/{total_events} ({(i+1)/total_events*100:.1f}%)")
            
            # Step 1: Determine category
            category = determine_category(event_row)
            result_df.at[idx, 'category'] = category
            
            # Step 2: Generate title
            title = generate_llm_title(category, event_row)
            result_df.at[idx, 'title'] = title
            
            event_type = generate_llm_description(category, event_row, title)
            result_df.at[idx, 'event_type'] = event_type
            #result_df.at[idx, 'description'] = description
            
        except Exception as e:
            print(f"Error processing event at index {idx}: {str(e)}")
            # Continue with next event if one fails
            continue
    
    print("Event processing complete!")
    return result_df

result_df = process_events_dataframe(big_events_data.iloc[:3000,:])

Processing 10 events...
Processing event 1/10 (10.0%)
Processing event 10/10 (100.0%)
Event processing complete!


In [102]:
import math
import random
import pandas as pd
import time
import os
import json
from datetime import datetime
from litellm import completion


# Weather interpretation codes
WEATHER_CODES = {
    0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
    45: "Fog", 48: "Depositing rime fog", 51: "Light drizzle", 53: "Moderate drizzle",
    55: "Dense drizzle", 56: "Light freezing drizzle", 57: "Dense freezing drizzle",
    61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain", 66: "Light freezing rain",
    67: "Heavy freezing rain", 71: "Slight snow fall", 73: "Moderate snow fall",
    75: "Heavy snow fall", 77: "Snow grains", 80: "Slight rain showers",
    81: "Moderate rain showers", 82: "Violent rain showers", 85: "Slight snow showers",
    86: "Heavy snow showers", 95: "Thunderstorm", 96: "Thunderstorm with slight hail",
    99: "Thunderstorm with heavy hail"
}

EVENT_CATEGORIES = [
    'Music & Concerts', 'Food & Drink', 'Education & Learning', 'Sports & Fitness',
    'Arts & Culture', 'Business & Networking', 'Technology', 'Community & Causes',
    'Health & Wellness', 'Entertainment', 'Seasonal & Festivals', 'Immersive Experiences'
]

# Helper functions
def parse_datetime(time_value):
    """Parse datetime from various formats"""
    if pd.isna(time_value):
        return None
    try:
        if isinstance(time_value, str):
            return datetime.fromisoformat(time_value.replace('Z', '+00:00'))
        return time_value
    except (ValueError, AttributeError):
        return None

def format_time(time_value):
    """Format datetime to readable string"""
    dt = parse_datetime(time_value)
    if dt:
        return dt.strftime("%B %d, %Y at %I:%M %p")
    return "TBD"

def calculate_weather_bias(event_row):
    """Calculate how much the weather pushes events indoors (0-1 scale)"""
    bias = 0
    
    # Weather code check
    weather_code = event_row.get('weather_code')
    if weather_code is not None and not pd.isna(weather_code):
        # Precipitation or extreme weather codes increase indoor bias
        if weather_code in [3, 45, 48, 51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 71, 73, 75, 77, 80, 81, 82, 85, 86, 95, 96, 99]:
            bias = max(bias, 0.6)
    
    # Temperature extremes
    temperature = event_row.get('temperature_2m_mean')
    if temperature is not None and not pd.isna(temperature):
        if temperature < 10 or temperature > 30:
            bias = max(bias, 0.5)
    
    # Heavy precipitation
    precipitation = event_row.get('precipitation_sum')
    if precipitation is not None and not pd.isna(precipitation) and precipitation > 5:
        bias = max(bias, 0.7)
    
    # Strong winds
    wind_speed = event_row.get('wind_speed_10m_max')
    if wind_speed is not None and not pd.isna(wind_speed) and wind_speed > 20:
        bias = max(bias, 0.6)
        
    return bias

def categorize_by_words(word_counts, is_evening, is_weekend, weather_indoor_bias):
    """Determine category based on word patterns"""
    total_words = sum(word_counts)
    num_unique_words = sum(1 for count in word_counts if count > 0)
    max_count = max(word_counts) if word_counts else 0
    concentration = max_count / total_words if total_words > 0 else 0
    
    # High word diversity suggests educational/business events
    if num_unique_words > 15 and concentration < 0.15:
        if is_evening:
            return random.choice(['Business & Networking', 'Technology'])
        else:
            return random.choice(['Education & Learning', 'Business & Networking', 'Technology'])
    
    # High concentration suggests focused events
    if concentration > 0.3:
        if is_evening:
            if weather_indoor_bias > 0.5:
                return random.choice(['Entertainment', 'Arts & Culture'])
            else:
                return random.choice(['Music & Concerts', 'Entertainment'])
        else:
            if weather_indoor_bias > 0.5:
                return random.choice(['Arts & Culture', 'Education & Learning'])
            else:
                return random.choice(['Sports & Fitness', 'Community & Causes'])
    
    # Medium concentration with few words often suggests food or social events
    if 0.2 <= concentration <= 0.3 and num_unique_words < 10:
        if is_evening:
            return random.choice(['Food & Drink', 'Entertainment'])
        else:
            return random.choice(['Food & Drink', 'Health & Wellness'])
    
    # Default to weighted selection
    return weighted_category_selection(weather_indoor_bias)

def categorize_by_time_weather(is_evening, is_weekend, weather_indoor_bias):
    """Category determination based on time and weather when word data unavailable"""
    if is_evening:
        if weather_indoor_bias > 0.5:
            return random.choice(['Entertainment', 'Arts & Culture', 'Food & Drink'])
        else:
            return random.choice(['Music & Concerts', 'Entertainment', 'Food & Drink'])
    elif is_weekend:
        if weather_indoor_bias > 0.5:
            return random.choice(['Arts & Culture', 'Education & Learning', 'Health & Wellness'])
        else:
            return random.choice(['Sports & Fitness', 'Community & Causes', 'Seasonal & Festivals'])
    else:
        return random.choice(['Business & Networking', 'Technology', 'Education & Learning'])

def weighted_category_selection(weather_indoor_bias):
    """Select category based on weights adjusted for weather"""
    # Base weights
    weights = [0.15, 0.12, 0.08, 0.08, 0.10, 0.08, 0.06, 0.08, 0.08, 0.07, 0.05, 0.05]
    
    # Adjust for weather
    if weather_indoor_bias > 0.5:
        # Indices for indoor-friendly events
        indoor_indices = [1, 2, 4, 5, 6, 8, 9]  
        # Indices for outdoor events
        outdoor_indices = [0, 3, 7, 10, 11]
        
        # Boost indoor categories, reduce outdoor ones
        for idx in indoor_indices:
            weights[idx] *= 1.3
        for idx in outdoor_indices:
            weights[idx] *= 0.7
            
        # Normalize weights
        total = sum(weights)
        weights = [w/total for w in weights]
    
    return random.choices(EVENT_CATEGORIES, weights=weights, k=1)[0]

def determine_category(event_row):
    """Determine the most likely event category based on available data"""
    # Extract word count data
    word_counts = [event_row.get(col, 0) for col in event_row.index if col.startswith('c_') and col != 'c_other']
    word_counts = [count for count in word_counts if not pd.isna(count)]
    
    # Calculate attendance metrics
    yes_count = event_row.get('yes_count', 0)
    maybe_count = event_row.get('maybe_count', 0)
    invited_count = event_row.get('invited_count', 0)
    is_large_event = invited_count > 500 if not pd.isna(invited_count) else False
    
    # Parse datetime
    dt = parse_datetime(event_row.get('start_time'))
    is_evening = dt and dt.hour >= 18
    is_weekend = dt and dt.weekday() >= 5
    month = dt.month if dt else None
    
    # Calculate weather influence
    weather_indoor_bias = calculate_weather_bias(event_row)
    
    # Check for seasonal events first
    if month and random.random() < (0.4 if month in [11, 12] else 0.1):
        return 'Seasonal & Festivals'
        
    # Handle large events differently
    if is_large_event:
        if weather_indoor_bias < 0.3:
            return random.choice(['Music & Concerts', 'Sports & Fitness', 'Immersive Experiences'])
        else:
            return random.choice(['Technology', 'Entertainment', 'Business & Networking'])
    
    # Analyze word patterns if available
    if word_counts:
        return categorize_by_words(word_counts, is_evening, is_weekend, weather_indoor_bias)
    
    # Default categorization based on time and weather
    return categorize_by_time_weather(is_evening, is_weekend, weather_indoor_bias)

def format_weather_context(event_row):
    """Format weather data into readable context"""
    weather_code = event_row.get('weather_code')
    temperature = event_row.get('temperature_2m_mean')
    precipitation = event_row.get('precipitation_sum')
    wind_speed = event_row.get('wind_speed_10m_max')
    
    if weather_code is None or pd.isna(weather_code):
        return ""
        
    weather_desc = WEATHER_CODES.get(weather_code, '')
    
    weather_context = f"Weather forecast: {weather_desc}"
    
    if temperature is not None and not pd.isna(temperature):
        weather_context += f", {temperature:.1f}°C"
    
    if precipitation is not None and not pd.isna(precipitation) and precipitation > 0:
        weather_context += f", with {precipitation:.1f}mm precipitation expected"
    
    if wind_speed is not None and not pd.isna(wind_speed) and wind_speed > 10:
        weather_context += f", and winds up to {wind_speed:.1f}km/h"
        
    return weather_context

def get_location_string(event_row):
    """Format location data into readable string"""
    city = event_row.get('city', 'Local')
    state = event_row.get('state', '')
    return f"{city}, {state}" if state else city

def determine_event_setting(category, event_row, weather_desc):
    """Determine if an event is indoor or outdoor"""
    # Categories that are typically outdoor
    outdoor_categories = [
        'Sports & Fitness', 'Music & Concerts', 
        'Seasonal & Festivals', 'Community & Causes', 
        'Immersive Experiences'
    ]
    
    # Extract weather conditions
    weather_code = event_row.get('weather_code')
    precipitation = event_row.get('precipitation_sum')
    wind_speed = event_row.get('wind_speed_10m_max')
    temperature = event_row.get('temperature_2m_mean')
    
    # Check if weather is unsuitable for outdoor events
    bad_weather = False
    if weather_code is not None and not pd.isna(weather_code):
        if weather_code in [51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 71, 73, 75, 77, 80, 81, 82, 85, 86, 95, 96, 99]:
            bad_weather = True
    
    if precipitation is not None and not pd.isna(precipitation) and precipitation > 5:
        bad_weather = True
    
    if wind_speed is not None and not pd.isna(wind_speed) and wind_speed > 20:
        bad_weather = True
    
    if temperature is not None and not pd.isna(temperature) and (temperature < 5 or temperature > 35):
        bad_weather = True
    
    # Determine event type
    if category in outdoor_categories and not bad_weather:
        return "Outdoor"
    elif category not in outdoor_categories:
        return "Indoor"
    else:
        # This could be an LLM call in a real implementation
        return "Indoor"  # Default for ambiguous cases

# Rate limiting and retry handling for API calls
class RateLimitHandler:
    def __init__(self, requests_per_minute=60, retry_attempts=5, initial_backoff=2):
        self.requests_per_minute = requests_per_minute
        self.retry_attempts = retry_attempts
        self.initial_backoff = initial_backoff
        self.request_times = []
        
    def wait_if_needed(self):
        """Wait if we're exceeding rate limits"""
        now = time.time()
        # Remove timestamps older than 1 minute
        self.request_times = [t for t in self.request_times if now - t < 60]
        
        # Check if we're at capacity
        if len(self.request_times) >= self.requests_per_minute:
            # Calculate time to wait
            oldest = min(self.request_times)
            wait_time = 60 - (now - oldest) + 1  # Add 1 second buffer
            print(f"Rate limit reached. Waiting {wait_time:.1f} seconds before next request.")
            time.sleep(wait_time)
        
        # Record this request
        self.request_times.append(time.time())
    
    def call_with_retry(self, func, *args, **kwargs):
        """Call a function with retry logic"""
        backoff = self.initial_backoff
        for attempt in range(self.retry_attempts):
            try:
                self.wait_if_needed()
                return func(*args, **kwargs)
            except Exception as e:
                if "rate limit" in str(e).lower() or "too many requests" in str(e).lower():
                    # Rate limit error - wait and retry
                    wait_time = backoff * (2 ** attempt)
                    print(f"Rate limit hit. Retrying in {wait_time} seconds (attempt {attempt+1}/{self.retry_attempts})")
                    time.sleep(wait_time)
                else:
                    # Other error - raise after logging
                    print(f"Error in API call: {str(e)}")
                    raise
        
        # If we got here, all retries failed
        raise Exception(f"All {self.retry_attempts} retry attempts failed due to rate limiting")

# Initialize rate limit handler
rate_limiter = RateLimitHandler(requests_per_minute=50, retry_attempts=5, initial_backoff=2)

def llm_response(prompt, llm_model="gpt-4o-mini"):
    """Function to get response from language model with rate limiting"""
    def _make_api_call():
        response = completion(
            model=llm_model,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    
    # Use the rate limiter to make the API call with retries
    return rate_limiter.call_with_retry(_make_api_call)

def generate_llm_title(category, event_row):
    """Generate a title for an event using LLM with rate limiting"""
    location = get_location_string(event_row)
    
    # Parse datetime
    dt = parse_datetime(event_row.get('start_time'))
    
    # Time and season descriptions
    time_desc, season = '', ''
    if dt:
        hour = dt.hour
        month = dt.month
        
        # Time of day
        if 5 <= hour < 12: time_desc = "morning"
        elif 12 <= hour < 17: time_desc = "afternoon"
        elif 17 <= hour < 21: time_desc = "evening"
        else: time_desc = "night"
            
        # Season (adjusted for hemisphere)
        is_northern = event_row.get('lat', 0) > 0 if 'lat' in event_row and not pd.isna(event_row['lat']) else True
        
        if is_northern:
            if month in [12, 1, 2]: season = "winter"
            elif month in [3, 4, 5]: season = "spring"
            elif month in [6, 7, 8]: season = "summer"
            else: season = "fall"
        else:
            if month in [12, 1, 2]: season = "summer"
            elif month in [3, 4, 5]: season = "fall"
            elif month in [6, 7, 8]: season = "winter"
            else: season = "spring"
    
    # List of event title examples for few-shot learning
    events_titles = [
        "Holi In Berlin 2025 - Festival of Colors | Food Stalls | Bollywood Dance",
        "Berlin Chocolate Festival 2025",
        "REGGAE ON THE ROOF",
        "Sing Along New York (Hits der 90er & 2000er), 29.03.2025",
        "Hackaday Europe 2025",
        "Just Taylor - One Night All Eras // Festsaal Kreuzberg",
        "2025 BUSINESS & LEADERSHIP CONFERENCE (BLC)",
        "SPECIAL: Millennials After Dark Professional Networking @ BLVD Park",
        "ASGTG E-Commerce Amazon Sellers Event 11th Annual Ed Rosenberg Conference",
        "San Diego Big Business, Tech & Entrepreneur Professional Networking Soiree",
    ]
    # Format examples to show expected style
    formatted_examples = "\n".join([f"- {title}" for title in events_titles])

    # Updated Prompt
    title_prompt = f"""
    Generate a catchy, natural-sounding event title similar in style to the examples below.

    ### **Examples of Great Event Titles:**
    {formatted_examples}

    ### **Event Details:**
    - Category: {category}
    - Location: {location}
    - Time of day: {time_desc}
    - Season: {season}

    ### **Instructions:**
    - Your title **must resemble the style of the examples**.
    - **Do NOT** use a formulaic "Season + Location + Event" format.
    - Only randomly incorporate season or time of day for some events
    - **Use dynamic, creative, and engaging phrasing**.
    - **Incorporate real-world language & cultural references** like the examples.
    - **No unnecessary punctuation**, no explanations, just return the title.

    ### **Output Format:**
    - **Only the title**
    - **No colons (":") unless absolutely necessary**
    - **Use expressive, engaging language**
    """
    
    title = llm_response(prompt=title_prompt)
    return title.strip().strip('"\'').strip()

def get_event_type(category, event_row):
    """Generate a description for an event using LLM"""
    
    # Format weather information
    weather_context = format_weather_context(event_row)
    # Determine if the event is indoor or outdoor
    event_type = determine_event_setting(category, event_row, weather_context)

    return event_type

# Checkpoint management functions
class CheckpointManager:
    def __init__(self, checkpoint_dir="checkpoints", checkpoint_frequency=10):
        """Initialize checkpoint manager with directory and save frequency"""
        self.checkpoint_dir = checkpoint_dir
        self.checkpoint_frequency = checkpoint_frequency
        
        # Create checkpoint directory if it doesn't exist
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        
    def save_checkpoint(self, df, batch_id, current_index, total_events):
        """Save current progress to a checkpoint file"""
        # Create checkpoint filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        checkpoint_path = os.path.join(self.checkpoint_dir, f"checkpoint_{batch_id}_{timestamp}.csv")
        
        # Save dataframe to CSV
        df.to_csv(checkpoint_path, index=False)
        
        # Save metadata about the checkpoint
        metadata = {
            "batch_id": batch_id,
            "current_index": current_index,
            "total_events": total_events,
            "timestamp": timestamp,
            "progress_percentage": round((current_index + 1) / total_events * 100, 2)
        }
        
        metadata_path = os.path.join(self.checkpoint_dir, f"metadata_{batch_id}_{timestamp}.json")
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
            
        print(f"Checkpoint saved at index {current_index+1}/{total_events} ({metadata['progress_percentage']}%) to {checkpoint_path}")
        
    def find_latest_checkpoint(self, batch_id=None):
        """Find the latest checkpoint file for a given batch ID"""
        if not os.path.exists(self.checkpoint_dir):
            return None, None
            
        # List all metadata files
        metadata_files = [f for f in os.listdir(self.checkpoint_dir) if f.startswith("metadata_")]
        
        if batch_id is not None:
            # Filter by batch ID
            metadata_files = [f for f in metadata_files if f"_{batch_id}_" in f]
            
        if not metadata_files:
            return None, None
            
        # Sort by timestamp (newest first)
        metadata_files.sort(reverse=True)
        latest_metadata = metadata_files[0]
        
        # Load metadata
        metadata_path = os.path.join(self.checkpoint_dir, latest_metadata)
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
            
        # Find corresponding CSV file
        batch_id = metadata["batch_id"]
        timestamp = metadata["timestamp"]
        csv_path = os.path.join(self.checkpoint_dir, f"checkpoint_{batch_id}_{timestamp}.csv")
        
        if os.path.exists(csv_path):
            return csv_path, metadata
        else:
            print(f"Warning: Metadata found but CSV missing for {batch_id}_{timestamp}")
            return None, None

def process_events_dataframe(events_df, batch_size=100, resume_from_checkpoint=True, batch_id=None):
    """
    Process events dataframe with rate limiting, checkpoint saving, and resume capability.
    
    Parameters:
    -----------
    events_df : pandas.DataFrame
        DataFrame containing event data
    batch_size : int, default=100
        Number of events to process before saving a checkpoint
    resume_from_checkpoint : bool, default=True
        Whether to resume from the latest checkpoint if available
    batch_id : str, optional
        Unique identifier for this processing batch, defaults to current timestamp
        
    Returns:
    --------
    pandas.DataFrame
        Processed dataframe with additional columns
    """
    # Create batch ID if not provided
    if batch_id is None:
        batch_id = datetime.now().strftime("%Y%m%d%H%M%S")
        
    # Initialize checkpoint manager
    checkpoint_mgr = CheckpointManager(checkpoint_frequency=batch_size)
    
    # Check for existing checkpoint
    latest_checkpoint, metadata = None, None
    if resume_from_checkpoint:
        latest_checkpoint, metadata = checkpoint_mgr.find_latest_checkpoint(batch_id)
        
    # Load from checkpoint if available
    start_index = 0
    if latest_checkpoint and metadata:
        print(f"Resuming from checkpoint at {metadata['progress_percentage']}% completion")
        result_df = pd.read_csv(latest_checkpoint)
        start_index = metadata['current_index'] + 1
    else:
        # Create new result dataframe
        result_df = events_df.copy()
        result_df['category'] = None
        result_df['title'] = None
        result_df['event_type'] = None
    
    # Process events from the starting index
    total_events = len(events_df)
    
    print(f"Processing {total_events - start_index} events (starting from index {start_index})...")
    
    # Track successful and failed events
    successful_events = 0
    failed_events = 0
    
    try:
        for i in range(start_index, total_events):
            idx = events_df.index[i]
            event_row = events_df.iloc[i]
            
            try:
                # Print progress update
                if i % 10 == 0 or i == total_events - 1:
                    print(f"Processing event {i+1}/{total_events} ({(i+1)/total_events*100:.1f}%)")
                
                # Step 1: Determine category
                category = determine_category(event_row)
                result_df.at[idx, 'category'] = category
                
                # Step 2: Generate title with rate limiting
                title = generate_llm_title(category, event_row)
                result_df.at[idx, 'title'] = title
                
                # Step 3: Generate event type
                event_type = get_event_type(category, event_row)
                result_df.at[idx, 'event_type'] = event_type
                
                successful_events += 1
                
                # Save checkpoint at regular intervals
                if (i + 1) % batch_size == 0 or i == total_events - 1:
                    checkpoint_mgr.save_checkpoint(result_df, batch_id, i, total_events)
                    
            except Exception as e:
                print(f"Error processing event at index {i} (ID: {idx}): {str(e)}")
                failed_events += 1
                # Continue with next event if one fails
                
                # Save checkpoint on failure too
                if failed_events % 5 == 0:  # Save after every 5 failures
                    checkpoint_mgr.save_checkpoint(result_df, batch_id, i, total_events)
                    
    except KeyboardInterrupt:
        print("\nProcess interrupted by user. Saving checkpoint...")
        checkpoint_mgr.save_checkpoint(result_df, batch_id, i, total_events)
        print("Checkpoint saved. You can resume later using the same batch ID.")
    
    # Final stats
    print(f"Processing complete!")
    print(f"Successful events: {successful_events}")
    print(f"Failed events: {failed_events}")
    
    # Save final result
    final_path = os.path.join(checkpoint_mgr.checkpoint_dir, f"final_result_{batch_id}.csv")
    result_df.to_csv(final_path, index=False)
    print(f"Final results saved to {final_path}")
    
    return result_df



In [103]:
result_df = process_events_dataframe(
    big_events_data.iloc[:4000,:], 
    batch_size=50,
    resume_from_checkpoint=True
)

Processing 4000 events (starting from index 0)...
Processing event 1/4000 (0.0%)
Processing event 11/4000 (0.3%)
Processing event 21/4000 (0.5%)
Processing event 31/4000 (0.8%)
Processing event 41/4000 (1.0%)
Checkpoint saved at index 50/4000 (1.25%) to checkpoints/checkpoint_20250314162842_20250314_162918.csv
Processing event 51/4000 (1.3%)
Rate limit reached. Waiting 25.5 seconds before next request.
Rate limit reached. Waiting 1.2 seconds before next request.
Processing event 61/4000 (1.5%)
Processing event 71/4000 (1.8%)
Processing event 81/4000 (2.0%)
Processing event 91/4000 (2.3%)
Checkpoint saved at index 100/4000 (2.5%) to checkpoints/checkpoint_20250314162842_20250314_163024.csv
Processing event 101/4000 (2.5%)
Rate limit reached. Waiting 18.7 seconds before next request.
Rate limit reached. Waiting 1.0 seconds before next request.
Rate limit reached. Waiting 1.1 seconds before next request.
Rate limit reached. Waiting 1.1 seconds before next request.
Processing event 111/400

In [106]:
result_df.tail()

,event_id,start_time,city,lat,lng,c_1,c_2,c_3,c_4,c_5,...,no_count,total_users,weather_code,temperature_2m_mean,precipitation_sum,precipitation_hours,wind_speed_10m_max,category,title,event_type
3995,480878599,2012-09-28T14:00:00.003Z,Medan,3.599,98.674,0,0,0,1,0,...,11,429,61.0,25.612747,5.2,9.0,5.506941,Food & Drink,Medan Sips & Bites Autumn Festival,Indoor
3996,837024523,2012-12-16T01:00:00.003Z,Phoenix,33.470,-112.097,19,7,3,0,3,...,27,600,53.0,9.214334,0.5,1.0,11.525623,Arts & Culture,Phoenix Winter Arts Festival - Night of Light ...,Indoor
3997,1855250657,2012-11-30T05:30:00.003Z,Hollywood,34.099,-118.345,6,3,4,4,4,...,9,216,53.0,15.612248,3.2,11.0,10.440000,Education & Learning,Hollywood Morning Masterclass - Ignite Your Cr...,Indoor
3998,489036365,2012-12-15T02:30:00.003Z,Aurora,44.015,-79.414,0,1,0,1,1,...,1,54,3.0,-1.252417,0.0,0.0,17.654688,Entertainment,Aurora Nights: Winter Wonderland Festival | Li...,Indoor
3999,4101010860,2012-08-22T01:00:00.003Z,Toronto,43.649,-79.395,10,2,4,3,1,...,17,410,3.0,20.875751,0.0,0.0,12.313894,Education & Learning,Toronto Summer Nights Learning Fest,Indoor


In [113]:
result_df_2 = process_events_dataframe(
    big_events_data.iloc[4000:8000,:], 
    batch_size=50,
    resume_from_checkpoint=True
)

Processing 4000 events (starting from index 0)...
Processing event 1/4000 (0.0%)
Processing event 11/4000 (0.3%)
Processing event 21/4000 (0.5%)
Processing event 31/4000 (0.8%)
Processing event 41/4000 (1.0%)
Checkpoint saved at index 50/4000 (1.25%) to checkpoints/checkpoint_20250314194509_20250314_194547.csv
Processing event 51/4000 (1.3%)
Rate limit reached. Waiting 22.8 seconds before next request.
Rate limit reached. Waiting 2.5 seconds before next request.
Processing event 61/4000 (1.5%)
Rate limit reached. Waiting 1.0 seconds before next request.
Processing event 71/4000 (1.8%)
Processing event 81/4000 (2.0%)
Processing event 91/4000 (2.3%)
Checkpoint saved at index 100/4000 (2.5%) to checkpoints/checkpoint_20250314194509_20250314_194651.csv
Processing event 101/4000 (2.5%)
Rate limit reached. Waiting 17.7 seconds before next request.
Rate limit reached. Waiting 4.8 seconds before next request.
Processing event 111/4000 (2.8%)
Processing event 121/4000 (3.0%)
Processing event 13

In [114]:
result_df_3 = process_events_dataframe(
    big_events_data.iloc[8000:,:], 
    batch_size=50,
    resume_from_checkpoint=True
)

Processing 3627 events (starting from index 0)...
Processing event 1/3627 (0.0%)
Processing event 11/3627 (0.3%)
Processing event 21/3627 (0.6%)
Processing event 31/3627 (0.9%)
Processing event 41/3627 (1.1%)
Checkpoint saved at index 50/3627 (1.38%) to checkpoints/checkpoint_20250314225423_20250314_225459.csv
Processing event 51/3627 (1.4%)
Rate limit reached. Waiting 25.6 seconds before next request.
Processing event 61/3627 (1.7%)
Processing event 71/3627 (2.0%)
Processing event 81/3627 (2.2%)
Processing event 91/3627 (2.5%)
Checkpoint saved at index 100/3627 (2.76%) to checkpoints/checkpoint_20250314225423_20250314_225601.csv
Processing event 101/3627 (2.8%)
Rate limit reached. Waiting 24.3 seconds before next request.
Rate limit reached. Waiting 1.1 seconds before next request.
Processing event 111/3627 (3.1%)
Processing event 121/3627 (3.3%)
Processing event 131/3627 (3.6%)
Processing event 141/3627 (3.9%)
Checkpoint saved at index 150/3627 (4.14%) to checkpoints/checkpoint_20250

In [115]:
result_df_main = pd.concat([result_df,result_df_2,result_df_3], axis=0)
result_df_main.head()

,event_id,start_time,city,lat,lng,c_1,c_2,c_3,c_4,c_5,...,total_users,weather_code,temperature_2m_mean,precipitation_sum,precipitation_hours,wind_speed_10m_max,category,title,event_type,weather_description
0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,0,3,1,1,1,...,522,51.0,28.318502,0.100000,1.0,10.972620,Education & Learning,Sihanoukville Knowledge Festival 2025 - Mornin...,Indoor,Light Drizzle
1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,5,3,3,1,3,...,1445,3.0,18.417002,0.000000,0.0,9.826088,Business & Networking,Palo Alto Summer Night Networking Bash,Indoor,Cloudy
2,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,5,3,3,1,3,...,1049,3.0,28.837250,0.000000,0.0,30.421598,Entertainment,Karachi Nights: Summer Fiesta Under the Stars,Indoor,Cloudy
3,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,0,0,0,0,0,...,555,61.0,24.851418,7.200001,12.0,6.989936,Arts & Culture,Funky Art Walk LA - Morning Vibes & Visual Del...,Indoor,Light Rain
4,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,0,0,0,0,0,...,107,3.0,10.835751,0.000000,0.0,10.495713,Arts & Culture,Palo Alto Art After Dark - A Fall Fiesta of Cr...,Indoor,Cloudy


In [116]:
len(result_df_main)

11627

In [117]:
import json

with open('/home/nkama/masters_thesis_project/thesis/descriptions.json', 'r') as f:
    data = json.load(f)

print(data)

{'0': {'day': {'description': 'Sunny', 'image': 'http://openweathermap.org/img/wn/01d@2x.png'}, 'night': {'description': 'Clear', 'image': 'http://openweathermap.org/img/wn/01n@2x.png'}}, '1': {'day': {'description': 'Mainly Sunny', 'image': 'http://openweathermap.org/img/wn/01d@2x.png'}, 'night': {'description': 'Mainly Clear', 'image': 'http://openweathermap.org/img/wn/01n@2x.png'}}, '2': {'day': {'description': 'Partly Cloudy', 'image': 'http://openweathermap.org/img/wn/02d@2x.png'}, 'night': {'description': 'Partly Cloudy', 'image': 'http://openweathermap.org/img/wn/02n@2x.png'}}, '3': {'day': {'description': 'Cloudy', 'image': 'http://openweathermap.org/img/wn/03d@2x.png'}, 'night': {'description': 'Cloudy', 'image': 'http://openweathermap.org/img/wn/03n@2x.png'}}, '45': {'day': {'description': 'Foggy', 'image': 'http://openweathermap.org/img/wn/50d@2x.png'}, 'night': {'description': 'Foggy', 'image': 'http://openweathermap.org/img/wn/50n@2x.png'}}, '48': {'day': {'description': '

In [118]:
with open('/home/nkama/masters_thesis_project/thesis/descriptions.json', 'r') as f:
    data = json.load(f)

# Create a mapping dictionary
weather_code_mapping = {}
for key, value in data.items():
  try:
    weather_code_mapping[float(key)] = value["day"]["description"]
  except KeyError:
      print(f"KeyError for key: {key}")  
  except ValueError:
      print(f"ValueError for key: {key}")
     

# Replace weather codes with descriptions
result_df_main['weather_description'] = result_df_main['weather_code'].map(weather_code_mapping)

# Example: Display the updated DataFrame
result_df_main.head()

,event_id,start_time,city,lat,lng,c_1,c_2,c_3,c_4,c_5,...,total_users,weather_code,temperature_2m_mean,precipitation_sum,precipitation_hours,wind_speed_10m_max,category,title,event_type,weather_description
0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,0,3,1,1,1,...,522,51.0,28.318502,0.100000,1.0,10.972620,Education & Learning,Sihanoukville Knowledge Festival 2025 - Mornin...,Indoor,Light Drizzle
1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,5,3,3,1,3,...,1445,3.0,18.417002,0.000000,0.0,9.826088,Business & Networking,Palo Alto Summer Night Networking Bash,Indoor,Cloudy
2,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,5,3,3,1,3,...,1049,3.0,28.837250,0.000000,0.0,30.421598,Entertainment,Karachi Nights: Summer Fiesta Under the Stars,Indoor,Cloudy
3,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,0,0,0,0,0,...,555,61.0,24.851418,7.200001,12.0,6.989936,Arts & Culture,Funky Art Walk LA - Morning Vibes & Visual Del...,Indoor,Light Rain
4,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,0,0,0,0,0,...,107,3.0,10.835751,0.000000,0.0,10.495713,Arts & Culture,Palo Alto Art After Dark - A Fall Fiesta of Cr...,Indoor,Cloudy


In [119]:
col = ['event_id', 'start_time', 'city', 'lat', 'lng','yes_count', 'maybe_count','invited_count', 'no_count',
       'total_users','weather_description', 'category', 'title', 'event_type']
result_df_main = result_df_main[col]
result_df_main.head()

,event_id,start_time,city,lat,lng,yes_count,maybe_count,invited_count,no_count,total_users,weather_description,category,title,event_type
0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,93,65,317,47,522,Light Drizzle,Education & Learning,Sihanoukville Knowledge Festival 2025 - Mornin...,Indoor
1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,162,112,1021,150,1445,Cloudy,Business & Networking,Palo Alto Summer Night Networking Bash,Indoor
2,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,22,8,993,26,1049,Cloudy,Entertainment,Karachi Nights: Summer Fiesta Under the Stars,Indoor
3,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,63,42,430,20,555,Light Rain,Arts & Culture,Funky Art Walk LA - Morning Vibes & Visual Del...,Indoor
4,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,41,12,53,1,107,Cloudy,Arts & Culture,Palo Alto Art After Dark - A Fall Fiesta of Cr...,Indoor


In [120]:
result_df_main.to_csv("complete_events_data.csv")

In [101]:
titles = result_df["title"].values
titles


array(['Sihanoukville Sunrise Networking Fall Fiesta',
       'Palo Alto Night of Tech Innovation - Summer Showcase & Networking Gala',
       'Karachi Summer Nights Networking Gala - Connect & Collaborate',
       'Cultural Vibes Morning Fest in LA - Artisans | Food Trucks | Live Beats',
       'Palo Alto Evening of Laughter - Comedy Under the Stars',
       'Winter Connect Sihanoukville - Networking Under the Stars',
       'Palo Alto Autumn Nights Festival - Fireworks & Food Trucks Under the Stars',
       'Palo Alto Night Run & Glow Fest 2025',
       'Siantar Tech Night: Innovations Under The Stars',
       'Spring Fiesta Jakarta 2025 - Celebrate Culture with Food and Music'],
      dtype=object)

In [72]:
import random

def refine_title(title):
    """Randomly modifies titles for additional variety."""
    transformations = [
        lambda x: x.upper(),  # Example: "Reggae On The Roof"
        lambda x: x.replace(":", " -"),  # "Morning Melodies: Fall Vibes" → "Morning Melodies - Fall Vibes"
        lambda x: x.replace(" in ", " | "),  # "Summer Nights Unplugged in Palo Alto" → "Summer Nights Unplugged | Palo Alto"
        lambda x: x.replace("&", "and"),  # Make formatting more natural
    ]
    
    if random.random() < 0.5:  # Apply transformation 50% of the time
        title = random.choice(transformations)(title)
    
    return title
ref_titles = []
for title in titles:
    ref_title =  refine_title(title)
    ref_titles.append(ref_title)

In [73]:
ref_titles

['Morning Minds: Embrace the Learning Wave | Sihanoukville',
 'Starlit Serenade: A Night of Music & Magic in Palo Alto',
 'Karachi Connect - Elevate Your Network Under the Stars',
 'Learning Under the LA Sky: Embrace the Fall Knowledge Festival',
 'Autumn Glow Festival: Enchanted Evenings in Palo Alto']

In [74]:
titles

array(['Morning Minds: Embrace the Learning Wave in Sihanoukville',
       'Starlit Serenade: A Night of Music & Magic in Palo Alto',
       'Karachi Connect: Elevate Your Network Under the Stars',
       'Learning Under the LA Sky: Embrace the Fall Knowledge Festival',
       'Autumn Glow Festival: Enchanted Evenings in Palo Alto'],
      dtype=object)

In [45]:
import ollama

In [2]:
import os

In [ ]:
import ollama


def llm_response(prompt, llm_model="deepseek-r1:1.5b"):
    # Initialize the Ollama client
    client = ollama.Client()
    response = client.generate(model="deepseek-r1:1.5b", prompt=prompt)
    return response.response

prompt = "What is Python?"   
response = llm_response(prompt=prompt)

print(response)
  

In [ ]:
import logging
# Configure logging to only show warnings and above
logging.getLogger("litellm").setLevel(logging.WARNING)
logging.getLogger().setLevel(logging.WARNING)
url = "http://localhost:11434/api/chat"
from litellm import completion
import os
import pandas as pd
import numpy as np
import random
from datetime import datetime
import string
#import openai  # Make sure to import openai for the GPT API calls
# First check if the API keys exist before setting them
openai_key = os.getenv("OPENAI_API_KEY")
groq_key = os.getenv("GROQ_API_KEY")

OPENAI_API_KEY=openai_key
GROQ_API_KEY=groq_key

# Function to load your events data
def load_events_data(file_path):
    events_df = pd.read_csv(file_path)
    return events_df

# Define event categories - these will be assigned based on patterns rather than specific words
EVENT_CATEGORIES = [
    'Music & Concerts',      # Covers live music, DJ performances, outdoor concerts
    'Food & Drink',          # Covers wine/beer tastings, food festivals, mixology, food truck rallies
    'Education & Learning',  # Covers workshops, educational events, book launches
    'Sports & Fitness',      # Covers sports viewing, fitness challenges
    'Arts & Culture',        # Covers art exhibits, photography, fashion, poetry, cultural celebrations
    'Business & Networking', # Covers networking, industry roundtables, product launches
    'Technology',            # Covers tech conferences, gaming tournaments
    'Community & Causes',    # Covers charity fundraisers, community gatherings, farmers markets
    'Health & Wellness',     # Covers wellness retreats, cooking workshops (health-focused)
    'Entertainment',         # Covers comedy, trivia, karaoke, film screenings
    'Seasonal & Festivals',  # Covers seasonal celebrations, street festivals, craft fairs
    'Immersive Experiences'  # Covers immersive events that don't fit other categories
]

# Function to determine category based on count patterns rather than specific words
def determine_category(event_row):
    """
    Determine event category based on frequency distribution patterns,
    time of year, and other event attributes
    """
    # Extract count values
    count_columns = [col for col in event_row.index if col.startswith('count_') and col != 'count_other']
    counts = [event_row[col] for col in count_columns]
    
    # Get time/date information if available
    start_time = event_row.get('start_time', None)
    month = None
    hour = None
    
    if start_time and not pd.isna(start_time):
        try:
            dt = datetime.fromisoformat(start_time.replace('Z', '+00:00'))
            month = dt.month  # 1-12
            hour = dt.hour    # 0-23
        except (ValueError, AttributeError):
            pass
    
    # Check if it might be a seasonal event
    seasonal_months = {
        'winter': [12, 1, 2],
        'spring': [3, 4, 5],
        'summer': [6, 7, 8],
        'fall': [9, 10, 11],
        'holiday': [11, 12]  # November, December for holiday events
    }
    
    # Special dates - could be expanded with more specific holiday checks
    is_seasonal_event = False
    if month:
        if month in seasonal_months['holiday']:
            is_seasonal_event = random.random() < 0.4  # 40% chance if in holiday months
        elif month in seasonal_months['summer']:
            is_seasonal_event = random.random() < 0.3  # 30% chance in summer
        else:
            is_seasonal_event = random.random() < 0.1  # 10% chance in other seasons
    
    # Evening events are more likely to be entertainment, music, food & drink
    is_evening_event = hour >= 18 if hour is not None else False
    
    # If it's determined to be a seasonal event
    if is_seasonal_event:
        return 'Seasonal & Festivals'
    
    # If we have no counts data, use time-based heuristics
    if not counts:
        if is_evening_event:
            return random.choice([
                'Music & Concerts', 
                'Entertainment', 
                'Food & Drink',
                'Arts & Culture'
            ])
        else:
            return random.choice(EVENT_CATEGORIES)
    
    # Analyze patterns in the counts
    total_words = sum(counts) + event_row.get('count_other', 0)
    num_unique_words = sum(1 for count in counts if count > 0)
    max_count = max(counts) if counts else 0
    concentration = max_count / total_words if total_words > 0 else 0
    
    # Heuristic categorization based on word patterns and time
    if num_unique_words > 15 and concentration < 0.15:
        # Diverse vocabulary, lower concentration
        if is_evening_event:
            return random.choice([
                'Business & Networking', 
                'Technology', 
                'Community & Causes',
                'Immersive Experiences'
            ])
        else:
            return random.choice([
                'Education & Learning', 
                'Business & Networking', 
                'Technology',
                'Community & Causes'
            ])
    elif concentration > 0.3:
        # High concentration on few words
        if is_evening_event:
            return random.choice([
                'Music & Concerts', 
                'Entertainment', 
                'Immersive Experiences'
            ])
        else:
            return random.choice([
                'Sports & Fitness', 
                'Music & Concerts',
                'Community & Causes'
            ])
    elif 0.2 <= concentration <= 0.3 and num_unique_words < 10:
        # Medium concentration, fewer unique words
        if is_evening_event:
            return random.choice([
                'Food & Drink', 
                'Arts & Culture', 
                'Entertainment'
            ])
        else:
            return random.choice([
                'Food & Drink', 
                'Arts & Culture', 
                'Health & Wellness'
            ])
    else:
        # Default to random selection with probability weights
        # Updated weights for all 12 categories
        return random.choices(
            EVENT_CATEGORIES,
            weights=[0.15, 0.12, 0.08, 0.08, 0.10, 0.08, 0.06, 0.08, 0.08, 0.07, 0.05, 0.05],
            k=1
        )[0]

# Create a function to format the time
def format_time(time_str):
    """Format ISO time string to readable time"""
    if pd.isna(time_str):
        return "TBD"
    
    try:
        dt = datetime.fromisoformat(time_str.replace('Z', '+00:00'))
        return dt.strftime("%A, %B %d at %-I:%M %p")  # e.g., "Monday, January 15 at 7:30 PM"
    except:
        return "TBD"

def llm_response(prompt, llm_model="llama3-8b-8192"):
    response = completion(
        #model=f"groq/{llm_model}", 
        model ='gpt-4o-mini',
        messages=[
            {"role": "user", "content": prompt}
        ],
    )
    return response.choices[0].message.content

# def clean_llm_response(response):
#     """Clean up LLM response to extract only the title"""
#     # Remove thinking process
#     if "<think>" in response:
#         response = response.split("</think>")[-1]
    
#     # Clean up the text
#     response = (response
#                .strip()  # Remove leading/trailing whitespace
#                .strip('.')  # Remove trailing periods
#                .strip('"\'')  # Remove quotes
#                .strip())  # Remove any remaining whitespace
    
#     # If response has multiple lines, take the last non-empty line
#     lines = [line.strip() for line in response.split('\n') if line.strip()]
#     if lines:
#         response = lines[-1]
    
#     return response



# def clean_llm_response(response):
#     """Clean up LLM response to extract only the title"""
#     if "<think>" in response:
#         response = response.split("</think>")[-1]
    
#     # Clean up the text
#     response = (response
#                .strip()  # Remove leading/trailing whitespace
#                .strip('.')  # Remove trailing periods
#                .strip('"\'')  # Remove quotes
#                .strip())  # Remove any remaining whitespace
    
    # # If response has multiple lines, take the last non-empty line
    # lines = [line.strip() for line in response.split('\n') if line.strip()]
    # if lines:
    #     response = lines[-1]
    
    # return response

# def llm_response(model="llama3.2", prompt=prompt):
#     # Initialize the Ollama client
#     client = ollama.Client()
#     response = client.generate(model="deepseek-r1:1.5b", prompt=prompt)
#     return clean_llm_response(response.response)

# def llm_response(model="deepseek-r1:1.5b", prompt=prompt):
#     """Get clean response from LLM without thinking process"""
#     # Initialize the Ollama client
#     client = ollama.Client()
#     response = client.generate(model="deepseek-r1:1.5b", prompt=prompt)
    
#     # Clean up the response
#     raw_response = response.response
    
#     # Remove thinking process and other artifacts
#     cleaned_response = raw_response.replace("<think>", "").replace("</think>", "")
    
#     # Remove everything between angle brackets
#     import re
#     cleaned_response = re.sub(r'<[^>]+>', '', cleaned_response)
    
#     # Remove "Alright, " and similar starter phrases
#     starters = ["Alright, ", "Let me ", "I need to ", "I should ", "Let's ", "I will "]
#     for starter in starters:
#         cleaned_response = cleaned_response.replace(starter, "")
    
#     # Clean up extra whitespace and newlines
#     cleaned_response = (cleaned_response
#                        .replace("\n", " ")
#                        .strip()
#                        .replace("  ", " "))
    
#     return cleaned_response

def generate_llm_title(category, event_row):
    """Generate event title using an LLM"""
    city = event_row.get('city', 'Local')
    state = event_row.get('state', '')
    location = f"{city}, {state}" if state else city
    
    # Create a prompt for title generation
    title_prompt = f"""
    Generate a catchy, attention-grabbing title for a '{category}' event in {location}.

    Guidelines:
    - Keep it concise (4-7 words)
    - Make it memorable and distinctive
    - Capture the essence of the category
    - Use creative language that evokes emotion
    - No generic placeholders or templates

    The event could be similar to one of these event types:
    ['Live music', 'DJ performances', 'Dancing', 'Karaoke', 
    'Comedy shows', 'Trivia nights', 'Art exhibits', 'Film screenings', 
    'Wine tasting', 'Craft beer tasting', 'Mixology classes', 'Food festivals',
    'Sports viewing parties', 'Networking events', 'Tech conferences',
    'Cultural celebrations', 'Outdoor concerts', 'Poetry slams', 'Book launches',
    'Fitness challenges', 'Cooking workshops', 'Wellness retreats',
    'Photography exhibitions', 'Fashion shows', 'Gaming tournaments',
    'Charity fundraisers', 'Farmers markets', 'Craft fairs',
    'Food truck rallies', 'Seasonal celebrations', 'Street festivals',
    'Immersive experiences', 'Industry roundtables', 'Product launches',
    'Educational workshops', 'Community gatherings']

    IMPORTANT: Return ONLY the title without any explanation, without quotes, explanation, 
    or any additional text or thinking process. ONLY return the title! .
    """
    
    # Generate title using LLM
    title = llm_response(prompt=title_prompt)
    
    # Clean up any quotation marks or extra formatting that might be returned
    title = title.strip().strip('"\'').strip()
    return title


# Function to generate event description and infer indoor/outdoor using GPT
def generate_llm_description(category, event_row, title):
    """Generate event description first, then infer event type (Indoor/Outdoor)."""
    city = event_row.get('city', 'Local')
    state = event_row.get('state', '')
    location = f"{city}, {state}" if state else city
    time_info = event_row.get('start_time', 'TBD')

    # Step 1: Generate event description using GPT
    description_prompt = f"""
    Generate a short and engaging 1-2 sentence event description based on the following details:

    Event Title: {title}
    Category: {category}
    Location: {location}
    Start Time: {time_info}

    The description should highlight the main features and excitement of the event.
    IMPORTANT: Return ONLY the description without any explanation or thinking process.
    """

    generated_description = llm_response(prompt=description_prompt)


    # Step 2: Infer Indoor/Outdoor classification using GPT
    classification_prompt = f"""
    Based on the following event description, determine whether this event is 'Indoor' or 'Outdoor':

    "{generated_description}"

    IMPORTANT: Please return only 'Indoor' or 'Outdoor' as the response without any explanation or thinking process.
    """
    event_type = llm_response(prompt=classification_prompt)

    return event_type, generated_description



# Main function to synthesize event details
def synthesize_event_details(events_df):
    """
    Synthesize event titles, descriptions, and categories without knowing actual word stems
    
    Parameters:
    - events_df: DataFrame containing the events data
    
    Returns:
    - DataFrame with added columns: 'category', 'title', 'description', 'event_type'
    """
    
    # Create new columns
    events_df = events_df.copy()
    
    # Initialize new columns using loc
    events_df.loc[:, 'category'] = ''
    events_df.loc[:, 'title'] = ''
    events_df.loc[:, 'description'] = ''
    events_df.loc[:, 'event_type'] = ''
    
    # Process each event
    for idx, event in events_df.iterrows():
        # Determine category based on patterns
        category = determine_category(event)
        events_df.loc[idx, 'category'] = category
        
        # Generate title using LLM
        title = generate_llm_title(category, event)
        events_df.loc[idx, 'title'] = title
        
        # Generate description and determine event type using GPT
        event_type, description = generate_llm_description(category, event, title)
        events_df.loc[idx, 'event_type'] = event_type
        events_df.loc[idx, 'description'] = description
        
        # Progress update for large datasets
        if idx % 1000 == 0:
            print(f"Processed {idx} events")
    
    return events_df



# Example usage
# 1. Basic synthetic generation:
# Example usage
# 1. Basic synthetic generation:
events_df = load_events_data('/home/nkama/masters_thesis_project/thesis/events_data.csv')
enriched_events = synthesize_event_details(events_df.iloc[:10,:])
# enriched_events = add_variety(enriched_events)
# enriched_events.to_csv('enriched_events.csv', index=False)


In [31]:
import logging
# Configure logging to only show warnings and above
logging.getLogger("litellm").setLevel(logging.WARNING)
logging.getLogger().setLevel(logging.WARNING)
import os
import signal
import sys
from litellm import completion
import os
import pandas as pd
import numpy as np
import random
from datetime import datetime
import string
import time
from tqdm import tqdm  # For progress tracking
from concurrent.futures import ThreadPoolExecutor, as_completed

# Add these imports at the top
import os
import signal
import sys

# Add these functions after your imports

def save_checkpoint(df, checkpoint_file, start_idx, current_idx):
    """Save the current progress to a checkpoint file"""
    checkpoint_data = {
        'last_processed_idx': current_idx,
        'start_idx': start_idx
    }
    
    # Save the dataframe with processed results so far
    df.to_csv(checkpoint_file + '.csv', index=False)
    
    # Save the checkpoint metadata
    with open(checkpoint_file + '.meta', 'w') as f:
        f.write(f"{checkpoint_data['start_idx']},{checkpoint_data['last_processed_idx']}")
    
    print(f"Checkpoint saved: processed up to index {current_idx}")

def load_checkpoint(checkpoint_file):
    """Load the last checkpoint if it exists"""
    if os.path.exists(checkpoint_file + '.meta') and os.path.exists(checkpoint_file + '.csv'):
        # Read metadata
        with open(checkpoint_file + '.meta', 'r') as f:
            data = f.read().strip().split(',')
            start_idx = int(data[0])
            last_idx = int(data[1])
        
        # Read saved dataframe
        saved_df = pd.read_csv(checkpoint_file + '.csv')
        
        print(f"Loaded checkpoint: resuming from index {last_idx + 1}")
        return saved_df, start_idx, last_idx + 1
    
    return None, None, None

def signal_handler(sig, frame):
    """Handle interruption signals by saving current progress"""
    print("\nInterruption detected! Saving checkpoint before exiting...")
    if 'current_events_df' in globals() and 'current_start_idx' in globals() and 'current_idx' in globals():
        save_checkpoint(current_events_df, 'events_checkpoint', current_start_idx, current_idx)
    print("Checkpoint saved. Exiting.")
    sys.exit(0)

#import openai  # Make sure to import openai for the GPT API calls
# First check if the API keys exist before setting them
openai_key = os.getenv("OPENAI_API_KEY")
groq_key = os.getenv("GROQ_API_KEY")

OPENAI_API_KEY=openai_key
GROQ_API_KEY=groq_key

# Function to load your events data
def load_events_data(file_path):
    events_df = pd.read_csv(file_path)
    return events_df

# Define event categories - these will be assigned based on patterns rather than specific words
EVENT_CATEGORIES = [
    'Music & Concerts',  
    'Food & Drink',    
    'Education & Learning', 
    'Sports & Fitness',     
    'Arts & Culture',        
    'Business & Networking', 
    'Technology',            
    'Community & Causes',   
    'Health & Wellness',     
    'Entertainment',         
    'Seasonal & Festivals', 
    'Immersive Experiences'  
]

# Function to determine category based on count patterns rather than specific words
def determine_category(event_row):
    """
    Determine event category based on frequency distribution patterns,
    time of year, and other event attributes
    """
    # Extract count values
    count_columns = [col for col in event_row.index if col.startswith('count_') and col != 'count_other']
    counts = [event_row[col] for col in count_columns]
    
    # Get time/date information if available
    start_time = event_row.get('start_time', None)
    month = None
    hour = None
    
    if start_time and not pd.isna(start_time):
        try:
            dt = datetime.fromisoformat(start_time.replace('Z', '+00:00'))
            month = dt.month  # 1-12
            hour = dt.hour    # 0-23
        except (ValueError, AttributeError):
            pass
    
    # Check if it might be a seasonal event
    seasonal_months = {
        'winter': [12, 1, 2],
        'spring': [3, 4, 5],
        'summer': [6, 7, 8],
        'fall': [9, 10, 11],
        'holiday': [11, 12]  # November, December for holiday events
    }
    
    # Special dates - could be expanded with more specific holiday checks
    is_seasonal_event = False
    if month:
        if month in seasonal_months['holiday']:
            is_seasonal_event = random.random() < 0.4  # 40% chance if in holiday months
        elif month in seasonal_months['summer']:
            is_seasonal_event = random.random() < 0.3  # 30% chance in summer
        else:
            is_seasonal_event = random.random() < 0.1  # 10% chance in other seasons
    
    # Evening events are more likely to be entertainment, music, food & drink
    is_evening_event = hour >= 18 if hour is not None else False
    
    # If it's determined to be a seasonal event
    if is_seasonal_event:
        return 'Seasonal & Festivals'
    
    # If we have no counts data, use time-based heuristics
    if not counts:
        if is_evening_event:
            return random.choice([
                'Music & Concerts', 
                'Entertainment', 
                'Food & Drink',
                'Arts & Culture'
            ])
        else:
            return random.choice(EVENT_CATEGORIES)
    
    # Analyze patterns in the counts
    total_words = sum(counts) + event_row.get('count_other', 0)
    num_unique_words = sum(1 for count in counts if count > 0)
    max_count = max(counts) if counts else 0
    concentration = max_count / total_words if total_words > 0 else 0
    
    # Heuristic categorization based on word patterns and time
    if num_unique_words > 15 and concentration < 0.15:
        # Diverse vocabulary, lower concentration
        if is_evening_event:
            return random.choice([
                'Business & Networking', 
                'Technology', 
                'Community & Causes',
                'Immersive Experiences'
            ])
        else:
            return random.choice([
                'Education & Learning', 
                'Business & Networking', 
                'Technology',
                'Community & Causes'
            ])
    elif concentration > 0.3:
        # High concentration on few words
        if is_evening_event:
            return random.choice([
                'Music & Concerts', 
                'Entertainment', 
                'Immersive Experiences'
            ])
        else:
            return random.choice([
                'Sports & Fitness', 
                'Music & Concerts',
                'Community & Causes'
            ])
    elif 0.2 <= concentration <= 0.3 and num_unique_words < 10:
        # Medium concentration, fewer unique words
        if is_evening_event:
            return random.choice([
                'Food & Drink', 
                'Arts & Culture', 
                'Entertainment'
            ])
        else:
            return random.choice([
                'Food & Drink', 
                'Arts & Culture', 
                'Health & Wellness'
            ])
    else:
        # Default to random selection with probability weights
        # Updated weights for all 12 categories
        return random.choices(
            EVENT_CATEGORIES,
            weights=[0.15, 0.12, 0.08, 0.08, 0.10, 0.08, 0.06, 0.08, 0.08, 0.07, 0.05, 0.05],
            k=1
        )[0]

# Create a function to format the time
def format_time(time_str):
    """Format ISO time string to readable time"""
    if pd.isna(time_str):
        return "TBD"
    
    try:
        dt = datetime.fromisoformat(time_str.replace('Z', '+00:00'))
        return dt.strftime("%A, %B %d at %-I:%M %p")  # e.g., "Monday, January 15 at 7:30 PM"
    except:
        return "TBD"

def llm_response(prompt, llm_model="llama3-8b-8192", max_retries=3, retry_delay=2):
    """
    Get response from LLM with retry logic for rate limits
    
    Parameters:
        prompt (str): The prompt text
        llm_model (str): The model to use
        max_retries (int): Maximum number of retries if rate limited
        retry_delay (int): Base delay between retries (will be exponentially increased)
    
    Returns:
        str: The LLM response
    """
    for attempt in range(max_retries + 1):
        try:
            response = completion(
                #model=f"groq/{llm_model}", 
                model='gpt-4o-mini',
                messages=[
                    {"role": "user", "content": prompt}
                ],
            )
            return response.choices[0].message.content
        except Exception as e:
            if "rate limit" in str(e).lower() and attempt < max_retries:
                # Exponential backoff
                sleep_time = retry_delay * (2 ** attempt)
                print(f"Rate limit hit, retrying in {sleep_time} seconds...")
                time.sleep(sleep_time)
            else:
                # If it's not a rate limit error or we've exhausted retries
                print(f"Error calling LLM API: {e}")
                return f"Error generating content: {e}"


def generate_llm_title(category, event_row):
    """Generate event title using an LLM"""
    city = event_row.get('city', 'Local')
    state = event_row.get('state', '')
    location = f"{city}, {state}" if state else city
    
    # Create a prompt for title generation
    title_prompt = f"""
    Generate a catchy, attention-grabbing title for a '{category}' event in {location}.

    Guidelines:
    - Keep it concise (10-20 words)
    - Capture the essence of the category
    - Use creative language that evokes emotion
    - No generic placeholders or templates

    The event could be or similar to one of these event types:
    ['Live music', 'DJ performances', 'Dancing', 'Karaoke', 
    'Comedy shows', 'Trivia nights', 'Art exhibits', 'Film screenings', 
    'Wine tasting', 'Craft beer tasting', 'Mixology classes', 'Food festivals',
    'Sports viewing parties', 'Networking events', 'Tech conferences',
    'Cultural celebrations', 'Outdoor concerts', 'Poetry slams', 'Book launches',
    'Fitness challenges', 'Cooking workshops', 'Wellness retreats',
    'Photography exhibitions', 'Fashion shows', 'Gaming tournaments',
    'Charity fundraisers', 'Farmers markets', 'Craft fairs',
    'Food truck rallies', 'Seasonal celebrations', 'Street festivals',
    'Immersive experiences', 'Industry roundtables', 'Product launches',
    'Educational workshops', 'Community gatherings']

    IMPORTANT: Return ONLY the title without any explanation, without quotes, explanation, 
    or any additional text or thinking process. ONLY return the title! .
    """
    
    # Generate title using LLM
    title = llm_response(prompt=title_prompt)
    
    # Clean up any quotation marks or extra formatting that might be returned
    title = title.strip().strip('"\'').strip()
    return title

def generate_llm_description(category, event_row, title):
    """Generate event description first, then infer event type (Indoor/Outdoor)."""
    city = event_row.get('city', 'Local')
    state = event_row.get('state', '')
    location = f"{city}, {state}" if state else city
    time_info = event_row.get('start_time', 'TBD')

    # Step 1: Generate event description using GPT
    description_prompt = f"""
    Generate a short and engaging 1-2 sentence event description based on the following details:

    Event Title: {title}
    Category: {category}
    Location: {location}
    Start Time: {time_info}

    The description should highlight the main features and excitement of the event.
    IMPORTANT: Return ONLY the description without any explanation or thinking process.
    """

    generated_description = llm_response(prompt=description_prompt)

    # Step 2: Infer Indoor/Outdoor classification using GPT
    classification_prompt = f"""
    Based on the following event description, determine whether this event is 'Indoor' or 'Outdoor':

    "{generated_description}"

    IMPORTANT: Please return only 'Indoor' or 'Outdoor' as the response without any explanation or thinking process.
    """
    event_type = llm_response(prompt=classification_prompt)

    return event_type, generated_description

# Function to process a single event
def process_single_event(idx, event):
    """Process a single event row to get category, title, description, and event type"""
    # Determine category based on patterns
    category = determine_category(event)
    
    # Generate title using LLM
    title = generate_llm_title(category, event)
    
    # Generate description and determine event type using GPT
    event_type, description = generate_llm_description(category, event, title)
    
    return {
        'idx': idx,
        'category': category,
        'title': title,
        'event_type': event_type,
        'description': description
    }

# Function to process batches with backoff for rate limits
def process_event_batch(event_batch, batch_size=5, max_workers=5):
    """
    Process a batch of events with parallel workers and rate limiting
    
    Parameters:
        event_batch: DataFrame batch of events to process
        batch_size: Size of sub-batches to further control rate limiting
        max_workers: Maximum number of parallel workers
    
    Returns:
        List of processed events with results
    """
    results = []
    
    # Process in smaller sub-batches to avoid rate limits
    for i in range(0, len(event_batch), batch_size):
        sub_batch = event_batch.iloc[i:i+batch_size]
        
        # Use ThreadPoolExecutor for parallel processing
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit tasks
            future_to_idx = {
                executor.submit(process_single_event, idx, event): idx 
                for idx, event in sub_batch.iterrows()
            }
            
            # Collect results as they complete
            for future in as_completed(future_to_idx):
                idx = future_to_idx[future]
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    print(f"Error processing event at index {idx}: {e}")
                    results.append({
                        'idx': idx,
                        'category': 'Error',
                        'title': f'Error: {str(e)}',
                        'event_type': 'Unknown',
                        'description': 'Failed to generate description'
                    })
        
        # Add delay between sub-batches to respect rate limits
        if i + batch_size < len(event_batch):
            time.sleep(1)  # 1 second pause between sub-batches
    
    return results

def synthesize_event_details(events_df, batch_size=10, worker_batch_size=5, max_workers=3, checkpoint_interval=5, checkpoint_file='events_checkpoint'):
    """
    Synthesize event titles, descriptions, and categories using batched API calls
    with checkpoint saving to prevent data loss on interruptions
    """
    global current_events_df, current_start_idx, current_idx
    
    # Check for existing checkpoint
    saved_df, start_idx, start_batch_idx = load_checkpoint(checkpoint_file)
    if saved_df is not None:
        events_df = saved_df
        start_position = start_batch_idx
        current_start_idx = start_idx
    else:
        # Create new columns if starting fresh
        events_df = events_df.copy()
        
        # Initialize new columns with proper data types
        events_df['category'] = pd.Series(dtype='object')  # Use object type for strings
        events_df['title'] = pd.Series(dtype='object')
        events_df['description'] = pd.Series(dtype='object')
        events_df['event_type'] = pd.Series(dtype='object')
        
        start_position = 0
        current_start_idx = 0
    
    # Set up current processing state for signal handler
    current_events_df = events_df
    
    # Register signal handlers
    signal.signal(signal.SIGINT, signal_handler)
    signal.signal(signal.SIGTERM, signal_handler)
    
    # Process in batches
    results = []
    
    total_to_process = len(events_df) - start_position
    print(f"Processing {total_to_process} events in batches of {batch_size}, starting from position {start_position}...")
    
    # Create batches
    checkpoint_counter = 0
    
    try:
        for i in tqdm(range(start_position, len(events_df), batch_size)):
            # Update current index for signal handler
            current_idx = i
            
            # Get the current batch
            batch = events_df.iloc[i:i+batch_size]
            
            # Process this batch
            batch_results = process_event_batch(
                batch, 
                batch_size=worker_batch_size,
                max_workers=max_workers
            )
            
            # Collect results
            results.extend(batch_results)
            
            # Update the DataFrame with collected results from this batch
            for result in batch_results:
                idx = result['idx']
                events_df.at[idx, 'category'] = result['category']  # Using .at instead of .loc
                events_df.at[idx, 'title'] = result['title']
                events_df.at[idx, 'description'] = result['description']
                events_df.at[idx, 'event_type'] = result['event_type']
            
            # Update the global dataframe for signal handler
            current_events_df = events_df
            
            # Save checkpoint periodically
            checkpoint_counter += 1
            if checkpoint_counter >= checkpoint_interval:
                save_checkpoint(events_df, checkpoint_file, current_start_idx, i + batch_size - 1)
                checkpoint_counter = 0
            
            # Delay between major batches to avoid rate limits
            if i + batch_size < len(events_df):
                time.sleep(2)  # 2 second pause between major batches
    
    except Exception as e:
        print(f"Error encountered: {e}")
        # Save on exception
        save_checkpoint(events_df, checkpoint_file, current_start_idx, current_idx)
        raise
    
    # Final save at completion
    save_checkpoint(events_df, checkpoint_file, current_start_idx, len(events_df) - 1)
    
    # Clean up checkpoint files when processing is complete
    if os.path.exists(checkpoint_file + '.meta'):
        os.remove(checkpoint_file + '.meta')
    
    return events_df

# Example usage
if __name__ == "__main__":
    # 1. Load data
    events_df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/filtered_events_data.csv")
    events_df = events_df.iloc[4000:7820,:] # Fix the typo here - remove the 'a'
    
    # 2. Process with batched API calls
    enriched_events = synthesize_event_details(
        events_df,
        batch_size=20,        # Process 20 events in each major batch
        worker_batch_size=5,  # Each worker handles 5 events at a time
        max_workers=3,        # Use 3 parallel workers
        checkpoint_interval=3 # Save every 3 batches
    )
    
    # 3. Save results
    enriched_events.to_csv('enriched_events_main_2.csv', index=False)
    
    # 4. Remove checkpoint files after successful completion
    # if os.path.exists('events_checkpoint.csv'):
    #     os.remove('events_checkpoint.csv')
    # print("Processing complete! Results saved to 'enriched_events_main.csv'")

Loaded checkpoint: resuming from index 1
Processing 3819 events in batches of 20, starting from position 1...


  0%|          | 0/191 [00:00<?, ?it/s]/tmp/ipykernel_20366/61490163.py:470: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Food & Drink' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  events_df.at[idx, 'category'] = result['category']  # Using .at instead of .loc
/tmp/ipykernel_20366/61490163.py:471: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Savor the Sounds: A Culinary Music Fest in Lawrence!' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  events_df.at[idx, 'title'] = result['title']
/tmp/ipykernel_20366/61490163.py:472: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Join us in Lawrence on December 2nd for "Savor the Sounds," a delightful culinary mus

Checkpoint saved: processed up to index 60


  3%|▎         | 5/191 [02:34<1:36:45, 31.21s/it]

Checkpoint saved: processed up to index 120


...

 62%|██████▏   | 119/191 [11:12:47<10:19:20, 516.12s/it]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

Rate limit hit, retrying in 2 seconds...

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

Rate limit hit, retrying in 4 seconds...

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

Rate limit hit, retrying in 2 seconds...

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

Rate limit hit, retrying in 2 seconds...

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

Rate limit hit, retrying in 2 seconds...

Give Feedback / Get Help: htt

 62%|██████▏   | 119/191 [11:21:19<6:52:13, 343.52s/it] 


SystemExit: 0

/home/nkama/.pyenv/versions/3.10.6/envs/thesisenv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [24]:
df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/enriched_events_main.csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,event_id,start_time,city,lat,lng,c_1,c_2,c_3,...,c_96,c_97,c_98,c_99,c_100,c_other,category,title,description,event_type
0,733,9120,3462970646,2012-10-31T00:00:00.001Z,Rio Rancho,35.241,-106.659,0,1,1,...,0,0,0,0,0,37,NaN,NaN,NaN,NaN
1,735,9311,3837329539,2012-11-25T03:00:00.003Z,Toronto,43.638,-79.424,0,0,0,...,0,0,0,0,0,33,Business & Networking,Connect & Conquer: Toronto's Ultimate Networki...,"Join us in Toronto for ""Connect & Conquer,"" th...",Indoor
2,736,9490,2509552492,2012-11-24T02:00:00.003Z,Medan,3.781,98.692,0,0,0,...,0,0,0,0,0,84,Seasonal & Festivals,"Medan's Enchanted Nights: A Festival of Light,...","Join us in Medan on November 24th for ""Medan's...",Outdoor
3,738,9581,1226009206,2012-09-23T04:00:00.003Z,Scottsdale,33.453,-111.926,2,0,1,...,0,0,0,0,0,95,Music & Concerts,Scottsdale Serenade: A Night of Rhythms and Re...,Join us for Scottsdale Serenade: A Night of Rh...,Outdoor
4,739,9760,2750134909,2012-11-02T16:30:00.003Z,Cape Town,-34.106,18.470,5,2,3,...,0,0,0,0,0,88,Health & Wellness,Revitalize: A Soulful Journey to Wellness and ...,"Join us in the heart of Cape Town for ""Revital...",Indoor


In [23]:
first_part = pd.read_csv("/home/nkama/masters_thesis_project/thesis/enriched_events_test.csv")
first_part.head()

,Unnamed: 0.1,Unnamed: 0,event_id,start_time,city,lat,lng,c_1,c_2,c_3,...,c_96,c_97,c_98,c_99,c_100,c_other,category,title,description,event_type
0,40,40,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,0,3,1,...,0,0,0,0,0,28,Seasonal & Festivals,Sunset Serenades: A Festival of Music and Magi...,Join us in picturesque Sihanoukville for Sunse...,Outdoor
1,51,51,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,5,3,3,...,1,0,0,0,0,103,Health & Wellness,"Revitalize and Thrive: A Day of Wellness, Musi...","Join us in Palo Alto for ""Revitalize and Thriv...",Indoor
2,74,74,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,5,3,3,...,1,0,0,0,0,103,Seasonal & Festivals,Karachi's Kaleidoscope: A Festive Fusion of So...,"Join us in Karachi for ""Karachi's Kaleidoscope...",Outdoor
3,155,156,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,0,0,0,...,0,0,0,0,0,6,Arts & Culture,"Echoes of Art: A Night of Music, Dance, and Cu...","Join us in Los Angeles for ""Echoes of Art: A N...",Indoor
4,177,178,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,0,0,0,...,0,0,0,0,0,0,Seasonal & Festivals,"Harvest Harmony: A Festival of Flavors, Fun, a...","Join us on November 11th in Palo Alto for ""Har...",Outdoor


In [16]:
df.isnull().sum().sum()

np.int64(8)

In [25]:
len(df), len(first_part)

(3950, 50)

In [27]:
enriched_events1 = pd.concat([first_part, df],axis=0)
enriched_events1.reset_index(drop=True)
len(enriched_events1)



4000

In [28]:
enriched_events1.head()

,Unnamed: 0.1,Unnamed: 0,event_id,start_time,city,lat,lng,c_1,c_2,c_3,...,c_96,c_97,c_98,c_99,c_100,c_other,category,title,description,event_type
0,40,40,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,0,3,1,...,0,0,0,0,0,28,Seasonal & Festivals,Sunset Serenades: A Festival of Music and Magi...,Join us in picturesque Sihanoukville for Sunse...,Outdoor
1,51,51,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,5,3,3,...,1,0,0,0,0,103,Health & Wellness,"Revitalize and Thrive: A Day of Wellness, Musi...","Join us in Palo Alto for ""Revitalize and Thriv...",Indoor
2,74,74,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,5,3,3,...,1,0,0,0,0,103,Seasonal & Festivals,Karachi's Kaleidoscope: A Festive Fusion of So...,"Join us in Karachi for ""Karachi's Kaleidoscope...",Outdoor
3,155,156,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,0,0,0,...,0,0,0,0,0,6,Arts & Culture,"Echoes of Art: A Night of Music, Dance, and Cu...","Join us in Los Angeles for ""Echoes of Art: A N...",Indoor
4,177,178,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,0,0,0,...,0,0,0,0,0,0,Seasonal & Festivals,"Harvest Harmony: A Festival of Flavors, Fun, a...","Join us on November 11th in Palo Alto for ""Har...",Outdoor


In [29]:
enriched_events1.to_csv("enriched_events_1.csv")

In [18]:
df[df["title"].isna()]

,Unnamed: 0.1,Unnamed: 0,event_id,start_time,city,lat,lng,c_1,c_2,c_3,...,c_96,c_97,c_98,c_99,c_100,c_other,category,title,description,event_type
0,733,9120,3462970646,2012-10-31T00:00:00.001Z,Rio Rancho,35.241,-106.659,0,1,1,...,0,0,0,0,0,37,NaN,NaN,NaN,NaN
21,774,14319,689246055,2012-10-12T02:00:00.003Z,Toronto,43.657,-79.404,5,3,2,...,0,0,0,0,0,52,NaN,NaN,NaN,NaN


In [6]:
df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/enriched_events_test.csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,event_id,start_time,city,lat,lng,c_1,c_2,c_3,...,c_96,c_97,c_98,c_99,c_100,c_other,category,title,description,event_type
0,40,40,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,0,3,1,...,0,0,0,0,0,28,Seasonal & Festivals,Sunset Serenades: A Festival of Music and Magi...,Join us in picturesque Sihanoukville for Sunse...,Outdoor
1,51,51,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,5,3,3,...,1,0,0,0,0,103,Health & Wellness,"Revitalize and Thrive: A Day of Wellness, Musi...","Join us in Palo Alto for ""Revitalize and Thriv...",Indoor
2,74,74,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,5,3,3,...,1,0,0,0,0,103,Seasonal & Festivals,Karachi's Kaleidoscope: A Festive Fusion of So...,"Join us in Karachi for ""Karachi's Kaleidoscope...",Outdoor
3,155,156,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,0,0,0,...,0,0,0,0,0,6,Arts & Culture,"Echoes of Art: A Night of Music, Dance, and Cu...","Join us in Los Angeles for ""Echoes of Art: A N...",Indoor
4,177,178,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,0,0,0,...,0,0,0,0,0,0,Seasonal & Festivals,"Harvest Harmony: A Festival of Flavors, Fun, a...","Join us on November 11th in Palo Alto for ""Har...",Outdoor


In [1]:
import pandas as pd
events_df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/filtered_events_data.csv")

In [2]:
events_df.head()

,Unnamed: 0.1,Unnamed: 0,event_id,start_time,city,lat,lng,c_1,c_2,c_3,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,40,40,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,0,3,1,...,0,0,0,0,0,0,0,0,0,28
1,51,51,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,5,3,3,...,0,0,0,0,1,0,0,0,0,103
2,74,74,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,5,3,3,...,0,0,0,0,1,0,0,0,0,103
3,155,156,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,0,0,0,...,0,0,0,0,0,0,0,0,0,6
4,177,178,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
len(events_df)

11627

In [ ]:
# Example usage
# 1. Basic synthetic generation:
#events_df = load_events_data('events.csv')
enriched_events = synthesize_event_details(events_df.iloc[:1000,:])
#enriched_events = add_variety(enriched_events)
# enriched_events.to_csv('enriched_events.csv', index=False)

In [61]:
enriched_events.head()

,Unnamed: 0.1,Unnamed: 0,event_id,start_time,city,lat,lng,c_1,c_2,c_3,...,c_96,c_97,c_98,c_99,c_100,c_other,category,title,description,event_type
0,40,40,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.5,0,3,1,...,0,0,0,0,0,28,Business & Networking,Bass Cat Gigglegram - Bling Your Business!,Bass Cat Gigglegram - Bling Your Business! Joi...,Indoor


In [22]:
print(df.description[4])
#enriched_events.to_csv("gpt_event:titles.csv")

Join us in the heart of Cape Town for "Revitalize: A Soulful Journey to Wellness and Connection," where rejuvenation meets community in a transformative experience designed to elevate your mind, body, and spirit. Immerse yourself in a series of uplifting workshops and holistic practices that will inspire you to reconnect with yourself and those around you.


In [6]:
events_df.head(2)

,Unnamed: 0.1,Unnamed: 0,event_id,start_time,city,lat,lng,c_1,c_2,c_3,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,40,40,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,0,3,1,...,0,0,0,0,0,0,0,0,0,28
1,51,51,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,5,3,3,...,0,0,0,0,1,0,0,0,0,103
